In [1]:
from bs4 import BeautifulSoup
import random
import pandas as pd
import requests
import time
import csv
import re
import sqlite3
import concurrent.futures
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.exceptions import HTTPError
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

# Define your list, or pass a list with pd.read_csv('filename.csv')

In [2]:
#passing a csv file into a list
etf_tickers = pd.read_csv('geo_etf.csv').dropna()
etf_tickers = etf_tickers['ticker'].values.tolist()
print(etf_tickers)
len(etf_tickers)

['BBJP', 'EWJ', 'EWJV', 'SCJ', 'FLJP', 'FLJH', 'GSJY', 'DFJ', 'DXJ', 'DXJS', 'JPXN', 'FLMX', 'EWW', 'EWZ', 'EPOL', 'EWM', 'EWT', 'FLTW', 'IEMG', 'CHIQ', 'FXI', 'KWEB', 'MCHI', 'FLKR', 'EWY', 'KORU', 'INDA', 'INDY', 'NFTY', 'EPI', 'GLIN', 'FLIN', 'SMIN', 'UAE', 'VNAM', 'VNM']


36

In [19]:
# Define your list of ETF tickers defined as etf_tickers
#etf_tickers = ['EWJ', 'SPY', 'XLK']
#etf_tickers = ['SCJ', 'EWJ', 'EWJV', 'FLJP']  # Add more tickers as needed
etf_tickers = ['CHIQ']
etf_tickers

['CHIQ']

# ETF Holdings

In [4]:
import asyncio
import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

# ETF Holdings code
async def etf_holdings(session, ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings'
    retries = 3

    for _ in range(retries):
        try:
            async with session.get(etf_holdings_url) as response:
                response.raise_for_status()
                soup = BeautifulSoup(await response.text(), 'html.parser')

                etf_name = etf_holdings_url.split('/')[-2]

                rows = soup.find_all('tr', {'class': 'holding'})

                stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
                stock_names = soup.find_all('td', {'data-th': 'Holding'})
                stock_weights = soup.find_all('td', {'data-th': '% Assets'})

                stock_symbols = [item.get_text() for item in stock_symbols]
                stock_names = [item.get_text() for item in stock_names]
                stock_weights = [item.get_text() for item in stock_weights]

                data = []
                
                for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
                    data.append([name, symbol, weight])

                df = pd.DataFrame(data, columns=["Holdings", "Symbol", "% Assets"])
                return df

        except (aiohttp.ClientError, asyncio.TimeoutError, ValueError) as e:
            print(f"Error fetching {etf_holdings_url}: {e}")
            print("Retrying...")
            await asyncio.sleep(random.uniform(0.0, 3.0))

    return None

async def main():
   # etf_tickers = ['CHIQ', 'FXI', 'CLOU', 'SKYY', 'SPY', 'IVV', 'VOO']  # Add your list of ETF tickers here
    etf_data_dict = {}
    not_scraped = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for ticker in etf_tickers:
            task = etf_holdings(session, ticker)
            tasks.append(task)

        results = await asyncio.gather(*tasks)

        for ticker, result in zip(etf_tickers, results):
            if result is not None:
                etf_data_dict[ticker] = result
            else:
                not_scraped.append(ticker)

    # Assign the result of pd.concat to the global variable
    global combined_data
   # global etf_data_dict
    combined_data = pd.concat(etf_data_dict, names=['Ticker'])
    
    if not_scraped:
        print("Tickers that were not successfully scraped:")
        print(not_scraped)

    print("Data saved.")
    print(combined_data)

# Run the event loop within the notebook environment
await main()

Data saved.
                                                    Holdings       Symbol  \
Ticker                                                                      
BBJP   0                                  Toyota Motor Corp.         7203   
       1                              Sony Group Corporation         6758   
       2                Mitsubishi UFJ Financial Group, Inc.         8306   
       3                                 Keyence Corporation         6861   
       4                              Mitsubishi Corporation         8058   
       5                                 Tokyo Electron Ltd.         8035   
       6               Sumitomo Mitsui Financial Group, Inc.         8316   
       7                           Shin-Etsu Chemical Co Ltd         4063   
       8                                        Hitachi,Ltd.         6501   
       9                          Recruit Holdings Co., Ltd.         6098   
       10                                   Mitsui & Co.,Ltd    

In [5]:
combined_data.to_csv('geo_etfholdings.csv')

# ETF INFO

In [6]:
#DONT TOUCH THIS CODE, it works
def get_etf_info(ticker):
    try:
        #construct the ETF URL
        etf_url = f'https://etfdb.com/etf/{ticker}/#etf-ticker-profile'
        etf_ticker = etf_url.split('/')[-2]
        response = requests.get(etf_url)
        soup = BeautifulSoup(response.text, 'html.parser')


        #Basic Information
        issuer = soup.find('span', text='Issuer').find_next_sibling('span').get_text()
        brand = soup.find('span', text='Brand').find_next_sibling('span').get_text()
        etf_home_page = soup.find('span', text='ETF Home Page').find_next_sibling('span').a['href']
        category = soup.find('span', text='Category').find_next_sibling('span').get_text(strip=True)
        asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()


        #weighting 
        td_elements = soup.find_all('td', class_='pull-right fs-table-alignment')
        #len(td_elements) >= 6
        sixth_element = td_elements[5]
        weighting = sixth_element.get_text(strip=True)


        expense_ratio = soup.find('span', text='Expense Ratio').find_next_sibling('span').get_text()

        aum = soup.find('span', text='AUM').find_next_sibling('span').get_text()
        shares_float = soup.find('span', text='Shares').find_next_sibling('span').get_text()
        one_mo_avg_volume = soup.find('span', text='1 Month Avg. Volume').find_next_sibling('span').get_text()
        three_mo_avg_volume = soup.find('span', text='3 Month Avg. Volume').find_next_sibling('span').get_text()

        #Dividends/Valuation
        etf_div = f'https://etfdb.com/etf/{ticker}/#etf-ticker-valuation-dividend'
        response1 = requests.get(etf_div)
        soup1 = BeautifulSoup(response1.text, 'html.parser')

        td_elements_div  = soup1.find_all('div', class_='text-center')
        pe_ratio = td_elements_div [1].get_text(strip=True)
        peer_pe_ratio = td_elements_div [3].get_text(strip=True)
        
        #Div Yield/Divvy
        divvy  = soup1.find_all('td', class_='center', attrs={'data-th':'Fund'})

        
        #Performance
        etf_perf_url  = f'https://etfdb.com/etf/{ticker}/#performance'
        response2 = requests.get(etf_perf_url)
        soup2 = BeautifulSoup(response2.text, 'html.parser')
        
#         etf_perf = soup2.find_all('td', {'data-th': 'Fund'}) 
#         etf_peer = soup2.find_all('td', {'data-th': 'ETF Database Category Average'}) 
        performance = soup2.find_all('td', class_='col-lg-4 col-xs-4')


# print(issuer, brand, etf_url, category, asset_class_size, weighting, one_mo_avg_volume, three_mo_avg_volume, expense_ratio, sep="\n")
# Create a dictionary with variable names and their values
        data_dict = {
            'ETF Ticker': etf_ticker,
            'Issuer': issuer,
            'Brand': brand,
            'ETF Home Page': etf_home_page,
            'Category': category,
            'Asset Class Size': asset_class_size,
            'Weighting': weighting,
            'Expense Ratio': expense_ratio,

            'PE Ratio' : pe_ratio,
            'Peer PE Ratio' : peer_pe_ratio,
            'ETF Dividend' : divvy[8].get_text(strip=True),
            'ETF Div Yield' : divvy[11].get_text(strip=True),
            'ETF Peer Dividend' : divvy[10].get_text(strip=True),
            'ETF Peer Div Yield' : divvy[11].get_text(strip=True),   

            #perf
            'YTD Return': performance[6].get_text(strip=True),
            '1 Month Return': performance[0].get_text(strip=True),
            '3 Month Return': performance[3].get_text(strip=True),
            '1 Year Return': performance[9].get_text(strip=True),
            '3 Year Return': performance[12].get_text(strip=True),
            '5 Year Return': performance[15].get_text(strip=True),
            
            'Peer YTD Return': performance[7].get_text(strip=True),
            'Peer 1 Month Return': performance[1].get_text(strip=True),
            'Peer 3 Month Return': performance[4].get_text(strip=True),
            'Peer 1 Year Return': performance[10].get_text(strip=True),
            'Peer 3 Year Return': performance[13].get_text(strip=True),
            'Peer 5 Year Return': performance[16].get_text(strip=True),

            'AUM': aum,
            'Shares float': shares_float,
            '1 Month Avg. Volume': one_mo_avg_volume,
            '3 Month Avg. Volume': three_mo_avg_volume,

        }

        return data_dict
    except AttributeError as attr_err:
        print(f"Attribute Error for ticker {ticker}: {attr_err}")
    except HTTPError as http_err:
        if http_err.response.status_code == 404:
            print(f"Skipping ticker {ticker}: Ticker not found.")
        else:
            print(f"Skipping ticker {ticker}: HTTP Error - {http_err}")
    except Exception as ex:
        print(f"Skipping ticker {ticker}: An error occurred - {ex}")
    return None


'''
#ETF Holdings code
def etf_holdings(ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings'
    retries = 3  # Number of retries
    for _ in range(retries):
        try:
            response = requests.get(etf_holdings_url)
            response.raise_for_status()  # Raise exception for bad responses
            soup = BeautifulSoup(response.text, 'html.parser')

            etf_name = etf_holdings_url.split('/')[-2]

            rows = soup.find_all('tr', {'class': 'holding'})

            stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
            stock_names = soup.find_all('td', {'data-th': 'Holding'})
            stock_weights = soup.find_all('td', {'data-th': '% Assets'})

            stock_symbols = [item.get_text() for item in stock_symbols]
            stock_names = [item.get_text() for item in stock_names]
            stock_weights = [item.get_text() for item in stock_weights]

            data = []

            for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
                data.append([name, symbol, weight])

            df = pd.DataFrame(data, columns=["holdings", "symbol", "% assets"])  
            return df                                                            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {etf_holdings_url}: {e}")
            print("Retrying...")
            time.sleep(random.uniform(0.0, 3.0))  # Adding a random sleep between retries

    return None  # Return None if scraping fails after retries


etf_data_dict = {}  # Dictionary to store ETF data
not_scraped = []  # List to store tickers that were not successfully scraped




#Execution for etf_holdings
# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the etf_holdings function for each ticker
    futures = [executor.submit(etf_holdings, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for ticker, future in zip(etf_tickers, as_completed(futures)):
        result = future.result()
        if result is not None:
            etf_data_dict[ticker] = result
        else:
            not_scraped.append(ticker)
        pbar.update(1)

        
# Create a DataFrame containing data from scraped ETFs
combined_data = pd.concat(etf_data_dict, names=['Ticker'])  #pd.concat(etf_data_dict.values(), keys=etf_data_dict.keys(), names=['Ticker'])

if not_scraped:
    print("Tickers that were not successfully scraped:")
    print(not_scraped)

print("Data saved.")
print(combined_data)


'''
results = []

#Execution for get_etf_info
# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the get_etf_info function for each ticker
    futures = [executor.submit(get_etf_info, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            results.append(result)
        pbar.update(1)
        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0, 3)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(results)
df

  3%|▎         | 1/36 [00:21<12:16, 21.04s/it]

Skipping ticker DXJS: An error occurred - HTTPSConnectionPool(host='etfdb.com', port=443): Max retries exceeded with url: /etf/DXJS/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000266B1F74AF0>, 'Connection to etfdb.com timed out. (connect timeout=None)'))Skipping ticker JPXN: An error occurred - HTTPSConnectionPool(host='etfdb.com', port=443): Max retries exceeded with url: /etf/JPXN/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000266B1F76920>, 'Connection to etfdb.com timed out. (connect timeout=None)'))
Skipping ticker FLJP: An error occurred - HTTPSConnectionPool(host='etfdb.com', port=443): Max retries exceeded with url: /etf/FLJP/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000266B34D0190>, 'Connection to etfdb.com timed out. (connect timeout=None)'))
Skipping ticker FLMX: An error occurred - HTTPSConnectionPool(host='etfdb.com', port=443): Max retries exceeded with url: 

100%|██████████| 36/36 [01:02<00:00,  1.74s/it]


""


""


In [38]:
combined_data.to_csv('etf_excl_bonds_info.csv')

In [43]:
# Display the DataFrame
df.sort_values('ETF Ticker')

,ETF Ticker,Issuer,Brand,ETF Home Page,Category,Asset Class Size,Weighting,Expense Ratio,PE Ratio,Peer PE Ratio,ETF Dividend,ETF Div Yield,ETF Peer Dividend,ETF Peer Div Yield,1 Month Return,3 Month Return,YTD Return,1 Year Return,3 Year Return,5 Year Return,Peer 1 Month Return,Peer 3 Month Return,Peer YTD Return,Peer 1 Year Return,Peer 3 Year Return,Peer 5 Year Return,AUM,Shares float,1 Month Avg. Volume,3 Month Avg. Volume
3,BBJP,JPMorgan Chase,JPMorgan,https://am.jpmorgan.com/us/en/asset-management...,Japan Equities,Large-Cap,Market Cap,0.19%,12.73,10.40,$ 0.68,1.37%,$ 0.68,1.37%,-4.87%,-1.63%,10.16%,6.31%,2.51%,2.59%,-1.86%,1.76%,15.58%,12.94%,7.99%,4.48%,"$8,386.1 M",169.9 M,"1,059,113","941,075"
27,CHIQ,"Mirae Asset Global Investments Co., Ltd.",Global X,http://www.globalxfunds.com/CHIQ,China Equities,Large-Cap,Market Cap,0.65%,16.58,9.49,$ 0.03,0.24%,$ 0.05,0.24%,-5.55%,0.05%,-9.53%,-11.00%,-10.53%,4.13%,-6.20%,-7.43%,-8.33%,-13.08%,-7.25%,-0.26%,$324.3 M,16.9 M,"110,096","106,363"
1,CLOU,"Mirae Asset Global Investments Co., Ltd.",Global X,https://www.globalxfunds.com/funds/clou/,Technology Equities,Multi-Cap,Market Cap,0.68%,35.33,13.63,$ 0.47,N/A,N/A,N/A,-8.06%,11.58%,20.21%,2.01%,-3.85%,N/A,-10.31%,5.59%,26.92%,-1.19%,2.78%,4.35%,$550.2 M,28.7 M,"253,387","254,122"
20,COPX,"Mirae Asset Global Investments Co., Ltd.",Global X,http://globalxfunds.com/COPX,Materials,Multi-Cap,Market Cap,0.65%,12.43,10.63,$ 0.51,2.48%,$ 0.90,2.48%,-8.68%,-3.16%,2.90%,17.47%,21.57%,14.01%,-7.78%,-3.69%,-0.47%,-0.58%,3.08%,5.31%,"$1,348.7 M",37.2 M,"596,838","527,495"
33,DIA,State Street,SPDR,https://www.spdrs.com/product/fund.seam?ticker...,Large Cap Growth Equities,Mega-Cap,Price,0.16%,16.82,15.16,$ 0.27,1.95%,$ 6.74,1.95%,-0.14%,3.68%,5.30%,3.41%,9.38%,8.23%,-4.49%,4.97%,15.76%,2.67%,5.87%,5.72%,"$29,525.6 M",85.5 M,"3,544,246","3,293,664"
14,EWJ,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Japan Equities,Large-Cap,Market Cap,0.50%,12.72,10.40,$ 0.43,0.94%,$ 0.56,0.94%,-5.01%,-1.85%,9.71%,5.74%,2.16%,2.35%,-1.86%,1.76%,15.58%,12.94%,7.99%,4.48%,"$13,080.6 M",221.0 M,"6,807,808","7,599,536"
7,EWJV,BlackRock Financial Management,iShares,https://www.ishares.com/us/products/307263/ish...,Japan Equities,Large-Cap,Market Cap,0.15%,9.08,10.40,$ 0.39,2.43%,$ 0.66,2.43%,-2.32%,1.97%,11.92%,12.55%,7.90%,N/A,-1.86%,1.76%,15.58%,12.94%,7.99%,4.48%,$212.6 M,7.9 M,"138,142","113,313"
18,EWT,BlackRock Financial Management,iShares,http://www.ishares.com/us/products/239686/isha...,Asia Pacific Equities,Large-Cap,Market Cap,0.57%,10.77,13.79,$ 7.56,16.66%,$ 7.56,16.66%,-7.86%,0.07%,12.35%,0.69%,6.43%,9.90%,-5.02%,0.10%,2.17%,-1.73%,2.61%,1.06%,"$3,362.0 M",74.1 M,"2,300,958","2,605,733"
0,FLJP,Franklin Templeton,Franklin,https://www.franklintempleton.com/investments/...,Japan Equities,Large-Cap,Market Cap,0.09%,13.56,10.40,$ 0.36,2.17%,$ 0.57,2.17%,-4.73%,-1.92%,9.59%,6.24%,2.56%,2.66%,-1.86%,1.76%,15.58%,12.94%,7.99%,4.48%,"$1,502.8 M",57.6 M,"427,987","473,794"
35,FXI,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,China Equities,Large-Cap,Market Cap,0.74%,10.62,9.49,$ 0.15,2.77%,$ 0.75,2.77%,-6.96%,-7.41%,-6.95%,-8.38%,-14.07%,-6.81%,-6.20%,-7.43%,-8.33%,-13.08%,-7.25%,-0.26%,"$5,056.3 M",187.4 M,"34,675,116","31,909,816"


In [44]:
#Save to csv
df.to_csv('etf_excl bonds_leveraged_holdings_info.csv', index=True)

In [37]:
df.count()

ETF Ticker             47
Issuer                 47
Brand                  47
ETF Home Page          47
Category               47
Asset Class Size       47
Weighting              47
Expense Ratio          47
PE Ratio               47
Peer PE Ratio          47
ETF Dividend           47
ETF Div Yield          47
ETF Peer Dividend      47
ETF Peer Div Yield     47
1 Month Return         47
3 Month Return         47
YTD Return             47
1 Year Return          47
3 Year Return          47
5 Year Return          47
Peer 1 Month Return    47
Peer 3 Month Return    47
Peer YTD Return        47
Peer 1 Year Return     47
Peer 3 Year Return     47
Peer 5 Year Return     47
AUM                    47
Shares float           47
1 Month Avg. Volume    47
3 Month Avg. Volume    47
dtype: int64

In [44]:
# Transpose data
df.T

,0
ETF Ticker,SCJ
Issuer,BlackRock Financial Management
Brand,iShares
ETF Home Page,http://us.ishares.com/product_info/fund/overvi...
Category,Japan Equities
Asset Class Size,Mid-Cap
Weighting,Market Cap
Expense Ratio,0.50%
PE Ratio,12.66
Peer PE Ratio,10.40


# Search in dataframe 'database' 

In [68]:
# Display the stored data for a specific ETF
selected_etf = 'IEMG'   # Change this
if selected_etf in combined_data.index.get_level_values('Ticker'):
    print(f"Holdings data for {selected_etf}:")
    print(combined_data.loc[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

No data found for IEMG.


# Make it look nicer

In [8]:
# Group the data by ETF
grouped_data = combined_data.groupby('Ticker')

# Iterate through each ETF group and display the data
for etf, group in grouped_data:
    print(f"Holdings data for {etf}:")
    print(group)
    print('\n')

Holdings data for BBJP:
                                        Holdings Symbol % Assets
Ticker                                                          
BBJP   0                      Toyota Motor Corp.   7203    5.65%
       1                  Sony Group Corporation   6758    2.70%
       2    Mitsubishi UFJ Financial Group, Inc.   8306    2.53%
       3                     Keyence Corporation   6861    2.07%
       4                  Mitsubishi Corporation   8058    1.88%
       5                     Tokyo Electron Ltd.   8035    1.76%
       6   Sumitomo Mitsui Financial Group, Inc.   8316    1.63%
       7               Shin-Etsu Chemical Co Ltd   4063    1.63%
       8                            Hitachi,Ltd.   6501    1.61%
       9              Recruit Holdings Co., Ltd.   6098    1.52%
       10                       Mitsui & Co.,Ltd   8031    1.48%
       11                     Itochu Corporation   8001    1.43%
       12                  Honda Motor Co., Ltd.   7267    1.43%
 

# Reshape it

In [9]:
import pandas as pd

# Assuming combined_data is your original DataFrame with the given data
# Reshape each ETF's Holdings data into a separate DataFrame
grouped_data = combined_data.groupby('Ticker')

# Create a list to store reshaped DataFrames
reshaped_dfs = []

# Iterate through each ETF
for ticker, section in grouped_data:
    section = section.reset_index(drop=True)
    section_holdings = section[['Holdings', 'Symbol', '% Assets']]
    section_holdings.columns = [f"{ticker}_{col}" for col in section_holdings.columns]
    reshaped_dfs.append(section_holdings)

# Concatenate the reshaped DataFrames
final_df = pd.concat(reshaped_dfs, axis=1)

# Display the reshaped DataFrame
final_df

,BBJP_Holdings,BBJP_Symbol,BBJP_% Assets,CHIQ_Holdings,CHIQ_Symbol,CHIQ_% Assets,DFJ_Holdings,DFJ_Symbol,DFJ_% Assets,DXJ_Holdings,DXJ_Symbol,DXJ_% Assets,DXJS_Holdings,DXJS_Symbol,DXJS_% Assets,EPI_Holdings,EPI_Symbol,EPI_% Assets,EPOL_Holdings,EPOL_Symbol,EPOL_% Assets,EWJ_Holdings,EWJ_Symbol,EWJ_% Assets,EWJV_Holdings,EWJV_Symbol,EWJV_% Assets,EWM_Holdings,EWM_Symbol,EWM_% Assets,EWT_Holdings,EWT_Symbol,EWT_% Assets,EWW_Holdings,EWW_Symbol,EWW_% Assets,EWY_Holdings,EWY_Symbol,EWY_% Assets,EWZ_Holdings,EWZ_Symbol,EWZ_% Assets,FLIN_Holdings,FLIN_Symbol,FLIN_% Assets,FLJH_Holdings,FLJH_Symbol,FLJH_% Assets,FLJP_Holdings,FLJP_Symbol,FLJP_% Assets,FLKR_Holdings,FLKR_Symbol,FLKR_% Assets,FLMX_Holdings,FLMX_Symbol,FLMX_% Assets,FLTW_Holdings,FLTW_Symbol,FLTW_% Assets,FXI_Holdings,FXI_Symbol,FXI_% Assets,GLIN_Holdings,GLIN_Symbol,GLIN_% Assets,GSJY_Holdings,GSJY_Symbol,GSJY_% Assets,IEMG_Holdings,IEMG_Symbol,IEMG_% Assets,INDA_Holdings,INDA_Symbol,INDA_% Assets,INDY_Holdings,INDY_Symbol,INDY_% Assets,JPXN_Holdings,JPXN_Symbol,JPXN_% Assets,KORU_Holdings,KORU_Symbol,KORU_% Assets,KWEB_Holdings,KWEB_Symbol,KWEB_% Assets,MCHI_Holdings,MCHI_Symbol,MCHI_% Assets,NFTY_Holdings,NFTY_Symbol,NFTY_% Assets,SCJ_Holdings,SCJ_Symbol,SCJ_% Assets,SMIN_Holdings,SMIN_Symbol,SMIN_% Assets,UAE_Holdings,UAE_Symbol,UAE_% Assets,VNAM_Holdings,VNAM_Symbol,VNAM_% Assets,VNM_Holdings,VNM_Symbol,VNM_% Assets
0,Toyota Motor Corp.,7203,5.65%,PDD Holdings Inc. Sponsored ADR Class A,PDD,9.68%,"Kobe Steel, Ltd.",5406,1.48%,Toyota Motor Corp.,7203,5.01%,"Kobe Steel, Ltd.",5406,1.47%,Tata Steel Limited,500470,6.58%,ORLEN Spolka Akcyjna,PKN,12.23%,Toyota Motor Corp.,7203,5.28%,Toyota Motor Corp.,7203,10.01%,Public Bank Bhd,1295,14.15%,"Taiwan Semiconductor Manufacturing Co., Ltd.",2330,22.42%,Grupo Financiero Banorte SAB de CV Class O,GFNORTEO,11.06%,"Samsung Electronics Co., Ltd.",005930,21.62%,Vale S.A.,VALE3,13.42%,Reliance Industries Limited,500325,8.34%,Toyota Motor Corp.,7203,4.99%,Toyota Motor Corp.,7203,5.03%,"Samsung Electronics Co., Ltd.",005930,17.34%,America Movil SAB de CV Class B,AMXB,11.46%,"Taiwan Semiconductor Manufacturing Co., Ltd.",2330,19.71%,Alibaba Group Holding Limited,9988,10.18%,MUTUAL FUND (OTHER),N/A,106.14%,Toyota Motor Corp.,7203,4.11%,"Taiwan Semiconductor Manufacturing Co., Ltd.",2330,5.33%,Reliance Industries Limited,500325,8.56%,HDFC Bank Limited,500180,13.46%,"Mitsubishi UFJ Financial Group, Inc.",8306,1.66%,iShares MSCI South Korea ETF,EWY,51.03%,PDD Holdings Inc. Sponsored ADR Class A,PDD,9.41%,Tencent Holdings Ltd.,700,12.74%,Cipla Ltd,500087,2.42%,Sojitz Corp.,2768,0.63%,U.S. Dollar,N/A,2.26%,Emirates Telecommunications Group Company PJSC,EAND,16.49%,Hoa Phat Group JSC,HPG,9.87%,Vingroup Joint Stock Company,VIC,7.70%
1,Sony Group Corporation,6758,2.70%,Alibaba Group Holding Limited,9988,9.09%,"Cosmo Energy Holdings Co., Ltd.",5021,0.82%,"Mitsubishi UFJ Financial Group, Inc.",8306,4.48%,"Tokyo Seimitsu Co., Ltd",7729,0.81%,Reliance Industries Limited,500325,5.90%,PKO Bank Polski S.A.,PKO,10.59%,Sony Group Corporation,6758,3.03%,"Mitsubishi UFJ Financial Group, Inc.",8306,5.02%,Malayan Banking Bhd.,1155,11.39%,"Hon Hai Precision Industry Co., Ltd.",2317,4.44%,Fomento Economico Mexicano SAB de CV Units Con...,FEMSAUBD,11.02%,SK hynix Inc.,000660,6.47%,Petroleo Brasileiro SA Pfd,PETR4,8.73%,Infosys Limited,500209,5.29%,Sony Group Corporation,6758,2.73%,Sony Group Corporation,6758,2.74%,SK hynix Inc.,000660,6.85%,Grupo Financiero Banorte SAB de CV Class O,GFNORTEO,10.79%,"Hon Hai Precision Industry Co., Ltd.",2317,5.16%,Meituan Class B,3690,9.33%,U.S. Dollar,N/A,-6.14%,"Mitsubishi UFJ Financial Group, Inc.",8306,2.25%,Tencent Holdings Ltd.,700,3.47%,ICICI Bank Limited,532174,5.73%,Reliance Industries Limited,500325,9.30%,"Recruit Holdings Co., Ltd.",6098,1.65%,U.S. Dollar,N/A,48.97%,Alibaba Group Holding Limited,9988,9.38%,Alibaba Group Holding Limited,9988,9.43%,NTPC Limited,532555,2.40%,Asics Corporation,7936,0.60%,IDFC First Ba

In [13]:
final_df.to_csv('geo_etf_holdings.csv')

# BOND ETF

In [24]:
#passing a csv file into a list
bond_etf_tickers = pd.read_csv('bond_holdings_tickers.csv').dropna()
bond_etf_tickers = bond_etf_tickers['ticker'].values.tolist()
print(bond_etf_tickers)
len(bond_etf_tickers)

['AGG', 'BIL', 'BND', 'BNDX', 'BSV', 'BWX', 'BWZ', 'EMB', 'FLIA', 'GOVT', 'HYG', 'IEF', 'IEMG', 'IGOV', 'IGSB', 'ISHG', 'JNK', 'LQD', 'MBB', 'MUB', 'SHV', 'SHY', 'TIP', 'TLT', 'VCIT', 'VCSH', 'VGIT', 'VGSH', 'VMBS', 'VTIP']


30

In [50]:
import asyncio
import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

# ETF Holdings code
async def etf_holdings(session, ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings'
    retries = 3

    for _ in range(retries):
        try:
            async with session.get(etf_holdings_url) as response:
                response.raise_for_status()
                soup = BeautifulSoup(await response.text(), 'html.parser')

                etf_name = etf_holdings_url.split('/')[-2]

                rows = soup.find_all('tr', {'class': 'holding'})

                stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
                stock_names = soup.find_all('td', {'data-th': 'Holding'})
                stock_weights = soup.find_all('td', {'data-th': '% Assets'})

                stock_symbols = [item.get_text() for item in stock_symbols]
                stock_names = [item.get_text() for item in stock_names]
                stock_weights = [item.get_text() for item in stock_weights]

                data = []
                
                for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
                    data.append([name, symbol, weight])

                df = pd.DataFrame(data, columns=["Holdings", "Symbol", "% Assets"])
                return df

        except (aiohttp.ClientError, asyncio.TimeoutError, ValueError) as e:
            print(f"Error fetching {etf_holdings_url}: {e}")
            print("Retrying...")
            await asyncio.sleep(random.uniform(0.0, 3.0))

    return None

async def main():
    global bond_combined_data
    global bond_etf_data_dict
   # etf_tickers = ['CHIQ', 'FXI', 'CLOU', 'SKYY', 'SPY', 'IVV', 'VOO']  # Add your list of ETF tickers here
    bond_etf_data_dict = {}
    not_scraped = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for ticker in bond_etf_tickers:
            task = etf_holdings(session, ticker)
            tasks.append(task)

        results = await asyncio.gather(*tasks)

        for ticker, result in zip(bond_etf_tickers, results):
            if result is not None:
                bond_etf_data_dict[ticker] = result
            else:
                not_scraped.append(ticker)

    bond_combined_data = pd.concat(bond_etf_data_dict.values(), keys=bond_etf_data_dict.keys(), names=['Ticker'])

    if not_scraped:
        print("Tickers that were not successfully scraped:")
        print(not_scraped)

    print("Data saved.")
    print(bond_combined_data)

# Run the event loop within the notebook environment
await main()

Data saved.
                                                    Holdings  Symbol % Assets
Ticker                                                                       
AGG    0            BlackRock Funds III Instl Money Mkt Fund   Other    7.41%
       1   GNMA II TBA 30yr 2% February Delivery 2.000% 0...     N/A    0.88%
       2     United States Treasury Notes 0.375% 31-JAN-2026     N/A    0.79%
       3     United States Treasury Notes 0.125% 31-JUL-2023     N/A    0.73%
       4      United States Treasury Bond 1.875% 15-FEB-2051     N/A    0.65%
       5   UMBS TBA 30yr 2.5% March Delivery 2.500% 01-OC...     N/A    0.63%
       6      United States Treasury Notes 0.25% 30-SEP-2023     N/A    0.59%
       7   UMBS TBA 15yr 1.5% February Delivery 1.500% 01...     N/A    0.58%
       8        United States Treasury Bond 2.0% 15-AUG-2051     N/A    0.55%
       9   UMBS TBA 30yr 2.5% February Delivery 2.500% 01...     N/A    0.52%
       10      United States Treasury Notes 1.5% 15-

In [51]:
bond_combined_data.to_csv('etf_bonds.csv')

# Reshape it

In [52]:
import pandas as pd

# Assuming combined_data is your original DataFrame with the given data
# Reshape each ETF's Holdings data into a separate DataFrame
bond_grouped_data = bond_combined_data.groupby('Ticker')

# Create a list to store reshaped DataFrames
bond_reshaped_dfs = []

# Iterate through each ETF
for ticker, section in bond_grouped_data:
    section = section.reset_index(drop=True)
    section_holdings = section[['Holdings', 'Symbol', '% Assets']]
    section_holdings.columns = [f"{ticker}_{col}" for col in section_holdings.columns]
    bond_reshaped_dfs.append(section_holdings)

# Concatenate the reshaped DataFrames
bond_holdings = pd.concat(bond_reshaped_dfs, axis=1)

# Display the reshaped DataFrame
bond_holdings

,AGG_Holdings,AGG_Symbol,AGG_% Assets,BIL_Holdings,BIL_Symbol,BIL_% Assets,BND_Holdings,BND_Symbol,BND_% Assets,BNDX_Holdings,BNDX_Symbol,BNDX_% Assets,BSV_Holdings,BSV_Symbol,BSV_% Assets,BWX_Holdings,BWX_Symbol,BWX_% Assets,BWZ_Holdings,BWZ_Symbol,BWZ_% Assets,EMB_Holdings,EMB_Symbol,EMB_% Assets,FLIA_Holdings,FLIA_Symbol,FLIA_% Assets,GOVT_Holdings,GOVT_Symbol,GOVT_% Assets,HYG_Holdings,HYG_Symbol,HYG_% Assets,IEF_Holdings,IEF_Symbol,IEF_% Assets,IEMG_Holdings,IEMG_Symbol,IEMG_% Assets,IGOV_Holdings,IGOV_Symbol,IGOV_% Assets,IGSB_Holdings,IGSB_Symbol,IGSB_% Assets,ISHG_Holdings,ISHG_Symbol,ISHG_% Assets,JNK_Holdings,JNK_Symbol,JNK_% Assets,LQD_Holdings,LQD_Symbol,LQD_% Assets,MBB_Holdings,MBB_Symbol,MBB_% Assets,MUB_Holdings,MUB_Symbol,MUB_% Assets,SHV_Holdings,SHV_Symbol,SHV_% Assets,SHY_Holdings,SHY_Symbol,SHY_% Assets,TIP_Holdings,TIP_Symbol,TIP_% Assets,TLT_Holdings,TLT_Symbol,TLT_% Assets,VCIT_Holdings,VCIT_Symbol,VCIT_% Assets,VCSH_Holdings,VCSH_Symbol,VCSH_% Assets,VGIT_Holdings,VGIT_Symbol,VGIT_% Assets,VGSH_Holdings,VGSH_Symbol,VGSH_% Assets,VMBS_Holdings,VMBS_Symbol,VMBS_% Assets,VTIP_Holdings,VTIP_Symbol,VTIP_% Assets
0,BlackRock Funds III Instl Money Mkt Fund,Other,7.41%,United States Treasury Bills 0.0% 07-SEP-2023,N/A,10.98%,U.S. Dollar,N/A,0.84%,Germany 0.0% 10-OCT-2025,N/A,0.45%,United States Treasury Notes 4.125% 31-JUL-2028,N/A,1.57%,TREASURY ISSUES (LONG-TERM),N/A,1.58%,Japan 0.1% 20-DEC-2025,N/A,1.97%,Government of Kuwait 3.5% 20-MAR-2027,N/A,0.75%,Germany 0.0% 15-AUG-2026,N/A,7.38%,United States Treasury Notes 1.375% 15-NOV-2031,N/A,4.97%,U.S. Dollar,N/A,0.69%,United States Treasury Notes 1.375% 15-NOV-2031,N/A,19.09%,"Taiwan Semiconductor Manufacturing Co., Ltd.",2330,5.33%,TREASURY ISSUES (LONG-TERM),N/A,0.39%,U.S. Dollar,N/A,0.30%,Government of Ireland 5.4% 13-MAR-2025,N/A,2.43%,State Street Institutional Liquid Reserves Fund,SSIXX,1.18%,BlackRock Cash Funds Treasury SL Agency Shares,N/A,0.75%,U.S. Dollar,N/A,8.66%,MUNICIPAL BONDS,N/A,0.83%,United States Treasury Bills 0.0% 05-OCT-2023,N/A,7.56%,United States Treasury Notes 1.5% 30-NOV-2024,N/A,6.14%,Government of the United States of America 0.1...,N/A,4.18%,United States Treasury Bond 1.875% 15-FEB-2051,N/A,9.95%,"T-Mobile USA, Inc. 3.875% 15-APR-2030",N/A,0.42%,Vanguard Cash Management Market Liquidity Fund,N/A,0.50%,United States Treasury Notes 2.625% 31-JUL-2029,N/A,2.76%,United States Treasury Notes 2.375% 15-AUG-2024,N/A,1.78%,U.S. Dollar,N/A,1.34%,United States Treasury Notes 0.625% 15-JAN-2024,N/A,5.81%
1,GNMA II TBA 30yr 2% February Delivery 2.000% 0...,N/A,0.88%,United States Treasury Bills 0.0% 12-OCT-2023,N/A,8.76%,United States Treasury Notes 0.75% 30-APR-2026,N/A,0.60%,U.S. Dollar,N/A,0.42%,United States Treasury Notes 1.5% 31-JAN-2027,N/A,0.77%,Government of China 3.25% 22-NOV-2028,N/A,1.43%,Japan 0.1% 20-MAR-2025,N/A,1.93%,U.S. Dollar,N/A,0.68%,France 2.0% 25-NOV-2032,N/A,4.79%,United States Treasury Notes 3.125% 15-NOV-2028,N/A,4.30%,"TransDigm, Inc. 6.25% 15-MAR-2026",N/A,0.47%,United States Treasury Notes 1.875% 15-FEB-2032,N/A,15.97%,Tencent Holdings Ltd.,700,3.47%,U.S. Dollar,N/A,0.23%,Bank of America Corporation 1.734% 22-JUL-2027,N/A,0.19%,Government of Ireland 1.0% 15-MAY-2026,N/A,2.18%,"TransDigm, Inc. 6.25% 15-MAR-2026",N/A,0.47%,Pfizer Investment Enterprises Pte. Ltd. 5.3% 1...,N/A,0.31%,FHLMC 30yr Pool#SD8146 2.000% 01-May-2051 2.0%...,N/A,1.27%,Atlanta Ga Wtr & Wastewtr Rev 5.0% 01-NOV-2040,N/A,0.23%,United States Treasury Bills 0.0% 02-NOV-2023,N/A,5.66%,United States Treasury Notes 4.5% 15-NOV-2025,N/A,5.27%,Government of the United States of America 0.6...,N/A,3.80%,United States Treasury Bond 2.0% 15-AUG-2051,N/A,7.28%,JPMorgan Chase & Co. 4.912% 25-JUL-2033,N/A,0.32%,United States Treasury Notes 4.125% 31-JUL-2028,N/A,0.40%,United States Treasury Notes 4.125% 15-NOV-2032,N/A,2.10%,United States Treasury Notes 0.375% 15-SEP-2024,N/A,1.67%,GNMA II 30yr Pool#MA6994 2.000% 20-Nov-2050 2....,N/A,0.59%,United States Treas

In [54]:
bond_holdings.to_csv('bond_holdings.csv')

# Search in dataframe 'database'

In [69]:
# Display the stored data for a specific ETF
selected_etf = 'IEMG'   # Change this
if selected_etf in bond_combined_data.index.get_level_values('Ticker'):
    print(f"Holdings data for {selected_etf}:")
    print(bond_combined_data.loc[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

Holdings data for IEMG:
                                             Holdings  Symbol % Assets
0        Taiwan Semiconductor Manufacturing Co., Ltd.    2330    5.33%
1                               Tencent Holdings Ltd.     700    3.47%
2                       Samsung Electronics Co., Ltd.  005930    3.16%
3                       Alibaba Group Holding Limited    9988    2.52%
4                                     Meituan Class B    3690    1.16%
5                         Reliance Industries Limited  500325    1.14%
6                                  ICICI Bank Limited  532174    0.76%
7                                     Infosys Limited  500209    0.68%
8                                   HDFC Bank Limited  500180    0.68%
9         China Construction Bank Corporation Class H     939    0.67%
10                                      SK hynix Inc.  000660    0.63%
11                                          Vale S.A.   VALE3    0.58%
12                               JD.com, Inc. Class A

# Make it look nicer

In [56]:
# Group the data by ETF
bond_grouped_data = bond_combined_data.groupby('Ticker')

# Iterate through each ETF group and display the data
for etf, group in bond_grouped_data:
    print(f"Holdings data for {etf}:")
    print(group)
    print('\n')

Holdings data for AGG:
                                                    Holdings Symbol % Assets
Ticker                                                                      
AGG    0            BlackRock Funds III Instl Money Mkt Fund  Other    7.41%
       1   GNMA II TBA 30yr 2% February Delivery 2.000% 0...    N/A    0.88%
       2     United States Treasury Notes 0.375% 31-JAN-2026    N/A    0.79%
       3     United States Treasury Notes 0.125% 31-JUL-2023    N/A    0.73%
       4      United States Treasury Bond 1.875% 15-FEB-2051    N/A    0.65%
       5   UMBS TBA 30yr 2.5% March Delivery 2.500% 01-OC...    N/A    0.63%
       6      United States Treasury Notes 0.25% 30-SEP-2023    N/A    0.59%
       7   UMBS TBA 15yr 1.5% February Delivery 1.500% 01...    N/A    0.58%
       8        United States Treasury Bond 2.0% 15-AUG-2051    N/A    0.55%
       9   UMBS TBA 30yr 2.5% February Delivery 2.500% 01...    N/A    0.52%
       10      United States Treasury Notes 1.5% 15-A

# BOND ETF INFO

In [28]:
#DONT TOUCH THIS CODE, it works
def get_etf_info(ticker):
    try:
        #construct the ETF URL
        etf_url = f'https://etfdb.com/etf/{ticker}/#etf-ticker-profile'
        etf_ticker = etf_url.split('/')[-2]
        response = requests.get(etf_url)
        soup = BeautifulSoup(response.text, 'html.parser')


        #Basic Information
        issuer = soup.find('span', text='Issuer').find_next_sibling('span').get_text()
        brand = soup.find('span', text='Brand').find_next_sibling('span').get_text()
        # etf_home_page = soup.find('span', text='ETF Home Page').find_next_sibling('span').a['href']
        category = soup.find('span', text='Category').find_next_sibling('span').get_text(strip=True)
        expense_ratio = soup.find('span', text='Expense Ratio').find_next_sibling('span').get_text()
        # asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()
        aum = soup.find('span', text='AUM').find_next_sibling('span').get_text()
        shares_float = soup.find('span', text='Shares').find_next_sibling('span').get_text()
        one_mo_avg_volume = soup.find('span', text='1 Month Avg. Volume').find_next_sibling('span').get_text()
        three_mo_avg_volume = soup.find('span', text='3 Month Avg. Volume').find_next_sibling('span').get_text()
        

        #Performance
        etf_perf_url  = f'https://etfdb.com/etf/{ticker}/#performance'
        response2 = requests.get(etf_perf_url)
        soup2 = BeautifulSoup(response2.text, 'html.parser')
        
#         etf_perf = soup2.find_all('td', {'data-th': 'Fund'}) 
#         etf_peer = soup2.find_all('td', {'data-th': 'ETF Database Category Average'}) 
        performance = soup2.find_all('td', class_='col-lg-4 col-xs-4')


# print(issuer, brand, etf_url, category, asset_class_size, weighting, one_mo_avg_volume, three_mo_avg_volume, expense_ratio, sep="\n")
# Create a dictionary with variable names and their values
        data_dict = {
            'ETF Ticker': etf_ticker,
            'Issuer': issuer,
            'Brand': brand,
           # 'ETF Home Page': etf_home_page,
            'Category': category,
            # 'Asset Class Size': asset_class_size,
            # 'Weighting': weighting,
            'Expense Ratio': expense_ratio,

            #perf
            'YTD Return': performance[6].get_text(strip=True),
            '1 Month Return': performance[0].get_text(strip=True),
            '3 Month Return': performance[3].get_text(strip=True),
            '1 Year Return': performance[9].get_text(strip=True),
            '3 Year Return': performance[12].get_text(strip=True),
            '5 Year Return': performance[15].get_text(strip=True),

            'Peer YTD Return': performance[7].get_text(strip=True),
            'Peer 1 Month Return': performance[1].get_text(strip=True),
            'Peer 3 Month Return': performance[4].get_text(strip=True),
            'Peer 1 Year Return': performance[10].get_text(strip=True),
            'Peer 3 Year Return': performance[13].get_text(strip=True),
            'Peer 5 Year Return': performance[16].get_text(strip=True),

            'AUM': aum,
            'Shares float': shares_float,
            '1 Month Avg. Volume': one_mo_avg_volume,
            '3 Month Avg. Volume': three_mo_avg_volume,

        }

        return data_dict
    except AttributeError as attr_err:
        print(f"Attribute Error for ticker {ticker}: {attr_err}")
    except HTTPError as http_err:
        if http_err.response.status_code == 404:
            print(f"Skipping ticker {ticker}: Ticker not found.")
        else:
            print(f"Skipping ticker {ticker}: HTTP Error - {http_err}")
    except Exception as ex:
        print(f"Skipping ticker {ticker}: An error occurred - {ex}")
    return None



results = []

# #Execution for get_etf_info
# # Create a ThreadPoolExecutor with max_workers equal to the number of tickers
# with ThreadPoolExecutor(max_workers=len(bond_etf_tickers)) as executor, tqdm(total=len(bond_etf_tickers)) as pbar:   ####################
#     # Submit the get_etf_info function for each ticker
#     futures = [executor.submit(get_etf_info, ticker) for ticker in bond_etf_tickers]   ####################
#     # Retrieve the results as they complete
#     for future in as_completed(futures):
#         result = future.result()
#         if result is not None:
#             results.append(result)
#         pbar.update(1)
#         # Introduce a random sleep interval between requests
#         sleep_interval = random.uniform(0, 3)  # Random sleep between 2 to 5 seconds
#         time.sleep(sleep_interval)


# Execution for get_etf_info
for ticker in tqdm(bond_etf_tickers):
    result = get_etf_info(ticker)
    if result is not None:
        results.append(result)
    # Introduce a random sleep interval between requests
    sleep_interval = random.uniform(0, 3)  # Random sleep between 2 to 5 seconds
    time.sleep(sleep_interval)


# Convert the list of dictionaries to a DataFrame
bond_df = pd.DataFrame(results)
bond_df

100%|██████████| 30/30 [03:26<00:00,  6.89s/it]


,ETF Ticker,Issuer,Brand,Category,Expense Ratio,YTD Return,1 Month Return,3 Month Return,1 Year Return,3 Year Return,5 Year Return,Peer YTD Return,Peer 1 Month Return,Peer 3 Month Return,Peer 1 Year Return,Peer 3 Year Return,Peer 5 Year Return,AUM,Shares float,1 Month Avg. Volume,3 Month Avg. Volume
0,AGG,BlackRock Financial Management,iShares,Total Bond Market,0.03%,0.74%,-0.72%,-1.73%,-1.51%,-4.65%,0.31%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$91,859.4 M",962.9 M,"8,069,644","7,688,148"
1,BIL,State Street,SPDR,Government Bonds,0.14%,3.25%,0.48%,1.35%,4.28%,1.50%,1.52%,0.36%,-0.75%,-2.08%,-1.71%,-2.86%,-0.06%,"$29,643.2 M",324.0 M,"7,001,861","6,362,734"
2,BND,Vanguard,Vanguard,Total Bond Market,0.03%,0.80%,-0.74%,-1.66%,-1.36%,-4.67%,0.36%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$94,236.0 M","1,332.5 M","5,141,600","5,457,397"
3,BNDX,Vanguard,Vanguard,Total Bond Market,0.07%,3.23%,-0.07%,-0.21%,0.43%,-3.67%,0.13%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$50,961.5 M","1,057.5 M","1,959,535","2,218,380"
4,BSV,Vanguard,Vanguard,Total Bond Market,0.04%,1.57%,-0.15%,-0.08%,0.98%,-1.59%,1.09%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$33,048.1 M",439.9 M,"2,146,135","3,067,663"
5,BWX,State Street,SPDR,International Government Bonds,0.35%,-2.17%,-3.30%,-2.94%,-0.11%,-9.09%,-3.57%,-2.23%,-2.28%,-2.13%,-1.97%,-6.00%,-2.46%,$862.2 M,39.8 M,"206,265","315,234"
6,BWZ,State Street,SPDR,International Government Bonds,0.35%,-2.53%,-2.79%,-1.67%,2.60%,-5.48%,-2.80%,-2.23%,-2.28%,-2.13%,-1.97%,-6.00%,-2.46%,$185.4 M,7.1 M,"30,978","30,981"
7,EMB,BlackRock Financial Management,iShares,Emerging Markets Bonds,0.39%,3.38%,-1.45%,0.45%,3.97%,-5.19%,0.16%,3.63%,-1.84%,-0.09%,5.31%,-2.55%,0.30%,"$14,234.3 M",169.4 M,"4,844,587","5,052,225"
8,FLIA,Franklin Templeton,Franklin,International Government Bonds,0.25%,1.85%,-0.53%,-0.49%,-0.54%,-2.31%,0.21%,-2.23%,-2.28%,-2.13%,-1.97%,-6.00%,-2.46%,$311.8 M,16.0 M,"52,083","48,167"
9,GOVT,BlackRock Financial Management,iShares,Government Bonds,0.05%,0.18%,-0.75%,-2.09%,-2.24%,-5.34%,-0.02%,0.36%,-0.75%,-2.08%,-1.71%,-2.86%,-0.06%,"$23,591.9 M","1,057.6 M","6,596,387","9,595,153"


In [29]:
bond_df

,ETF Ticker,Issuer,Brand,Category,Expense Ratio,YTD Return,1 Month Return,3 Month Return,1 Year Return,3 Year Return,5 Year Return,Peer YTD Return,Peer 1 Month Return,Peer 3 Month Return,Peer 1 Year Return,Peer 3 Year Return,Peer 5 Year Return,AUM,Shares float,1 Month Avg. Volume,3 Month Avg. Volume
0,AGG,BlackRock Financial Management,iShares,Total Bond Market,0.03%,0.74%,-0.72%,-1.73%,-1.51%,-4.65%,0.31%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$91,859.4 M",962.9 M,"8,069,644","7,688,148"
1,BIL,State Street,SPDR,Government Bonds,0.14%,3.25%,0.48%,1.35%,4.28%,1.50%,1.52%,0.36%,-0.75%,-2.08%,-1.71%,-2.86%,-0.06%,"$29,643.2 M",324.0 M,"7,001,861","6,362,734"
2,BND,Vanguard,Vanguard,Total Bond Market,0.03%,0.80%,-0.74%,-1.66%,-1.36%,-4.67%,0.36%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$94,236.0 M","1,332.5 M","5,141,600","5,457,397"
3,BNDX,Vanguard,Vanguard,Total Bond Market,0.07%,3.23%,-0.07%,-0.21%,0.43%,-3.67%,0.13%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$50,961.5 M","1,057.5 M","1,959,535","2,218,380"
4,BSV,Vanguard,Vanguard,Total Bond Market,0.04%,1.57%,-0.15%,-0.08%,0.98%,-1.59%,1.09%,2.18%,-0.42%,-0.42%,0.83%,-1.69%,0.46%,"$33,048.1 M",439.9 M,"2,146,135","3,067,663"
5,BWX,State Street,SPDR,International Government Bonds,0.35%,-2.17%,-3.30%,-2.94%,-0.11%,-9.09%,-3.57%,-2.23%,-2.28%,-2.13%,-1.97%,-6.00%,-2.46%,$862.2 M,39.8 M,"206,265","315,234"
6,BWZ,State Street,SPDR,International Government Bonds,0.35%,-2.53%,-2.79%,-1.67%,2.60%,-5.48%,-2.80%,-2.23%,-2.28%,-2.13%,-1.97%,-6.00%,-2.46%,$185.4 M,7.1 M,"30,978","30,981"
7,EMB,BlackRock Financial Management,iShares,Emerging Markets Bonds,0.39%,3.38%,-1.45%,0.45%,3.97%,-5.19%,0.16%,3.63%,-1.84%,-0.09%,5.31%,-2.55%,0.30%,"$14,234.3 M",169.4 M,"4,844,587","5,052,225"
8,FLIA,Franklin Templeton,Franklin,International Government Bonds,0.25%,1.85%,-0.53%,-0.49%,-0.54%,-2.31%,0.21%,-2.23%,-2.28%,-2.13%,-1.97%,-6.00%,-2.46%,$311.8 M,16.0 M,"52,083","48,167"
9,GOVT,BlackRock Financial Management,iShares,Government Bonds,0.05%,0.18%,-0.75%,-2.09%,-2.24%,-5.34%,-0.02%,0.36%,-0.75%,-2.08%,-1.71%,-2.86%,-0.06%,"$23,591.9 M","1,057.6 M","6,596,387","9,595,153"


In [30]:
bond_df.to_csv('bond_df.csv')

# Improve, to save into a SQLite database

In [ ]:
'''
# Store the ETF data in an SQLite3 database
db_filename = 'etf_holdings.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Create a table to store ETF data
create_table_query = '''
CREATE TABLE IF NOT EXISTS etf_holdings (
    id INTEGER PRIMARY KEY,
    ticker TEXT,
    holdings TEXT,
    symbol TEXT,
    percent_assets TEXT
)
'''
cursor.execute(create_table_query)

# Insert data into the table
for ticker, df in etf_data_dict.items():
    for _, row in df.iterrows():
        insert_query = '''
        INSERT INTO etf_holdings (ticker, holdings, symbol, percent_assets)
        VALUES (?, ?, ?, ?)
        '''
        cursor.execute(insert_query, (ticker, row['holdings'], row['symbol'], row['% assets']))

# Commit changes and close the connection
conn.commit()
conn.close()

# Display the stored data for a specific ETF
selected_etf = 'EWJ'
conn = sqlite3.connect(db_filename)
query = f'SELECT * FROM etf_holdings WHERE ticker = "{selected_etf}"'
selected_df = pd.read_sql_query(query, conn)
conn.close()

if not selected_df.empty:
    print(f"Holdings data for {selected_etf}:")
    print(selected_df)
else:
    print(f"No data found for {selected_etf}.")
    
'''

# Single pull

In [70]:
etf_tickers = ['EWZ']
etf_tickers

['EWZ']

# ETF Holdings

In [74]:
import asyncio
import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

global combined_data
global single_etf_dict

# ETF Holdings code
async def etf_holdings(session, ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings'
    retries = 3

    for _ in range(retries):
        try:
            async with session.get(etf_holdings_url) as response:
                response.raise_for_status()
                soup = BeautifulSoup(await response.text(), 'html.parser')

                etf_name = etf_holdings_url.split('/')[-2]

                rows = soup.find_all('tr', {'class': 'holding'})

                stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
                stock_names = soup.find_all('td', {'data-th': 'Holding'})
                stock_weights = soup.find_all('td', {'data-th': '% Assets'})

                stock_symbols = [item.get_text() for item in stock_symbols]
                stock_names = [item.get_text() for item in stock_names]
                stock_weights = [item.get_text() for item in stock_weights]

                data = []
                
                for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
                    data.append([name, symbol, weight])

                df = pd.DataFrame(data, columns=["Holdings", "Symbol", "% Assets"])
                return df

        except (aiohttp.ClientError, asyncio.TimeoutError, ValueError) as e:
            print(f"Error fetching {etf_holdings_url}: {e}")
            print("Retrying...")
            await asyncio.sleep(random.uniform(0.0, 3.0))

    return None

async def main():
   # etf_tickers = ['CHIQ', 'FXI', 'CLOU', 'SKYY', 'SPY', 'IVV', 'VOO']  # Add your list of ETF tickers here
    single_etf_dict = {}
    not_scraped = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for ticker in etf_tickers:
            task = etf_holdings(session, ticker)
            tasks.append(task)

        results = await asyncio.gather(*tasks)

        for ticker, result in zip(etf_tickers, results):
            if result is not None:
                single_etf_dict[ticker] = result
            else:
                not_scraped.append(ticker)

    # Assign the result of pd.concat to the global variable

    combined_data = pd.concat(single_etf_dict, names=['Ticker'])
    
    if not_scraped:
        print("Tickers that were not successfully scraped:")
        print(not_scraped)

    print("Data saved.")
    print(combined_data)

# Run the event loop within the notebook environment
await main()

Data saved.
                                               Holdings Symbol % Assets
Ticker                                                                 
EWZ    0                                      Vale S.A.  VALE3   12.37%
       1                     Petroleo Brasileiro SA Pfd  PETR4    8.31%
       2                   Itau Unibanco Holding SA Pfd  ITUB4    7.26%
       3                         Petroleo Brasileiro SA  PETR3    7.06%
       4                          Banco Bradesco SA Pfd  BBDC4    4.39%
       5                  B3 SA - Brasil, Bolsa, Balcao  B3SA3    4.35%
       6                                       Ambev SA  ABEV3    3.68%
       7                                       WEG S.A.  WEGE3    3.60%
       8                         Localiza Rent A Car SA  RENT3    3.42%
       9                           Banco do Brasil S.A.  BBAS3    2.52%
       10            Itausa - Investimentos Itau SA Pfd  ITSA4    2.51%
       11                                   Suzano S

# ETF INFO SINGLE

In [75]:
#DONT TOUCH THIS CODE, it works
def get_etf_info(ticker):
    try:
        #construct the ETF URL
        etf_url = f'https://etfdb.com/etf/{ticker}/#etf-ticker-profile'
        etf_ticker = etf_url.split('/')[-2]
        response = requests.get(etf_url)
        soup = BeautifulSoup(response.text, 'html.parser')


        #Basic Information
        issuer = soup.find('span', text='Issuer').find_next_sibling('span').get_text()
        brand = soup.find('span', text='Brand').find_next_sibling('span').get_text()
        etf_home_page = soup.find('span', text='ETF Home Page').find_next_sibling('span').a['href']
        category = soup.find('span', text='Category').find_next_sibling('span').get_text(strip=True)
        asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()


        #weighting 
        td_elements = soup.find_all('td', class_='pull-right fs-table-alignment')
        #len(td_elements) >= 6
        sixth_element = td_elements[5]
        weighting = sixth_element.get_text(strip=True)


        expense_ratio = soup.find('span', text='Expense Ratio').find_next_sibling('span').get_text()

        aum = soup.find('span', text='AUM').find_next_sibling('span').get_text()
        shares_float = soup.find('span', text='Shares').find_next_sibling('span').get_text()
        one_mo_avg_volume = soup.find('span', text='1 Month Avg. Volume').find_next_sibling('span').get_text()
        three_mo_avg_volume = soup.find('span', text='3 Month Avg. Volume').find_next_sibling('span').get_text()

        #Dividends/Valuation
        etf_div = f'https://etfdb.com/etf/{ticker}/#etf-ticker-valuation-dividend'
        response1 = requests.get(etf_div)
        soup1 = BeautifulSoup(response1.text, 'html.parser')

        td_elements_div  = soup1.find_all('div', class_='text-center')
        pe_ratio = td_elements_div [1].get_text(strip=True)
        peer_pe_ratio = td_elements_div [3].get_text(strip=True)
        
        #Div Yield/Divvy
        divvy  = soup1.find_all('td', class_='center', attrs={'data-th':'Fund'})

        
        #Performance
        etf_perf_url  = f'https://etfdb.com/etf/{ticker}/#performance'
        response2 = requests.get(etf_perf_url)
        soup2 = BeautifulSoup(response2.text, 'html.parser')
        
#         etf_perf = soup2.find_all('td', {'data-th': 'Fund'}) 
#         etf_peer = soup2.find_all('td', {'data-th': 'ETF Database Category Average'}) 
        performance = soup2.find_all('td', class_='col-lg-4 col-xs-4')


# print(issuer, brand, etf_url, category, asset_class_size, weighting, one_mo_avg_volume, three_mo_avg_volume, expense_ratio, sep="\n")
# Create a dictionary with variable names and their values
        data_dict = {
            'ETF Ticker': etf_ticker,
            'Issuer': issuer,
            'Brand': brand,
            'ETF Home Page': etf_home_page,
            'Category': category,
            'Asset Class Size': asset_class_size,
            'Weighting': weighting,
            'Expense Ratio': expense_ratio,

            'PE Ratio' : pe_ratio,
            'Peer PE Ratio' : peer_pe_ratio,
            'ETF Dividend' : divvy[8].get_text(strip=True),
            'ETF Div Yield' : divvy[11].get_text(strip=True),
            'ETF Peer Dividend' : divvy[10].get_text(strip=True),
            'ETF Peer Div Yield' : divvy[11].get_text(strip=True),   

            #perf
            '1 Month Return': performance[0].get_text(strip=True),
            '3 Month Return': performance[3].get_text(strip=True),
            'YTD Return': performance[6].get_text(strip=True),
            '1 Year Return': performance[9].get_text(strip=True),
            '3 Year Return': performance[12].get_text(strip=True),
            '5 Year Return': performance[15].get_text(strip=True),

            'Peer 1 Month Return': performance[1].get_text(strip=True),
            'Peer 3 Month Return': performance[4].get_text(strip=True),
            'Peer YTD Return': performance[7].get_text(strip=True),
            'Peer 1 Year Return': performance[10].get_text(strip=True),
            'Peer 3 Year Return': performance[13].get_text(strip=True),
            'Peer 5 Year Return': performance[16].get_text(strip=True),

            'AUM': aum,
            'Shares float': shares_float,
            '1 Month Avg. Volume': one_mo_avg_volume,
            '3 Month Avg. Volume': three_mo_avg_volume,

        }

        return data_dict
    except AttributeError as attr_err:
        print(f"Attribute Error for ticker {ticker}: {attr_err}")
    except HTTPError as http_err:
        if http_err.response.status_code == 404:
            print(f"Skipping ticker {ticker}: Ticker not found.")
        else:
            print(f"Skipping ticker {ticker}: HTTP Error - {http_err}")
    except Exception as ex:
        print(f"Skipping ticker {ticker}: An error occurred - {ex}")
    return None

results = []

#Execution for get_etf_info
# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the get_etf_info function for each ticker
    futures = [executor.submit(get_etf_info, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            results.append(result)
        pbar.update(1)
        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0, 3)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Convert the list of dictionaries to a DataFrame
df_single = pd.DataFrame(results)
df_single

100%|██████████| 1/1 [00:08<00:00,  8.41s/it]


,ETF Ticker,Issuer,Brand,ETF Home Page,Category,Asset Class Size,Weighting,Expense Ratio,PE Ratio,Peer PE Ratio,ETF Dividend,ETF Div Yield,ETF Peer Dividend,ETF Peer Div Yield,1 Month Return,3 Month Return,YTD Return,1 Year Return,3 Year Return,5 Year Return,Peer 1 Month Return,Peer 3 Month Return,Peer YTD Return,Peer 1 Year Return,Peer 3 Year Return,Peer 5 Year Return,AUM,Shares float,1 Month Avg. Volume,3 Month Avg. Volume
0,EWZ,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Latin America Equities,Large-Cap,Market Cap,0.58%,4.48,7.81,$ 0.76,8.99%,$ 2.78,8.99%,-3.33%,9.49%,16.93%,9.49%,10.93%,5.56%,-1.95%,8.87%,20.67%,17.06%,12.34%,4.32%,"$5,026.3 M",166.4 M,"22,997,416","25,817,048"


In [76]:
df_single.to_csv('df_single.csv')

# LEGACY IGNORE BELOW

In [11]:
print(performance[0].get_text(strip=True))
print(performance[3].get_text(strip=True))
print(performance[6].get_text(strip=True))
print(performance[9].get_text(strip=True))
print(performance[12].get_text(strip=True))
print(performance[15].get_text(strip=True))

-0.28%
3.08%
13.69%
10.16%
4.65%
2.67%


# Top 15 Holdings

In [4]:
etf_tickers

['SPY',
 'QQQ',
 'IWM',
 'DIA',
 'IYT',
 'SMH',
 'EWJ',
 'BBJP',
 'XLK',
 'XLE',
 'XLY',
 'XLI',
 'XLF',
 'XLC',
 'XLV',
 'XLP',
 'XLU',
 'XLB',
 'XLRE',
 'XPH',
 'XBI',
 'XRT',
 'XHB',
 'RTH',
 'IYC',
 'ITB',
 'EWT',
 'SKYY',
 'CLOU',
 'ROBO',
 'CHIQ',
 'KWEB',
 'DBA',
 'DBB',
 'USO',
 'UNG',
 'UGA',
 'WEAT',
 'CORN',
 'GLD',
 'SOYB',
 'CANE',
 'GLDM',
 'SLV',
 'GDX',
 'GDXJ',
 'NUGT',
 'JNUG',
 'DUST',
 'JDST',
 'PPLT',
 'COPX',
 'UUP',
 'FXY',
 'FXE',
 'FXB',
 'FXC',
 'VXX',
 'UVXY',
 'SVXY',
 'UVIX',
 'SVIX',
 'BITO',
 'BND',
 'AGG',
 'BNDX',
 'LQD',
 'HYG',
 'JNK',
 'TIP',
 'VTIP',
 'GOVT',
 'EMB',
 'IGSB',
 'IEF',
 'TLT',
 'IGOV',
 'BWX',
 'FLIA',
 'BWZ',
 'ISHG',
 'TYO',
 'TBF',
 'TMV',
 'TBT',
 'TYD',
 'UBT',
 'TMF',
 'TQQQ',
 'SQQQ',
 'SH',
 'SPXU',
 'SPXS',
 'SOXL',
 'SOXS',
 'TNA',
 'TZA',
 'FNGD',
 'YINN',
 'YANG',
 'BOIL',
 'UCO',
 'SCO',
 'AGQ']

In [9]:
import pandas as pd
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
from tqdm import tqdm  # Assuming you have tqdm installed

def scrape_etf_data(ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings'
    retries = 3  # Number of retries
    for _ in range(retries):
        try:
            response = requests.get(etf_holdings_url)
            response.raise_for_status()  # Raise exception for bad responses
            soup = BeautifulSoup(response.text, 'html.parser')

            etf_name = etf_holdings_url.split('/')[-2]

            rows = soup.find_all('tr', {'class': 'holding'})

            stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
            stock_names = soup.find_all('td', {'data-th': 'Holding'})
            stock_weights = soup.find_all('td', {'data-th': '% Assets'})

            stock_symbols = [item.get_text() for item in stock_symbols]
            stock_names = [item.get_text() for item in stock_names]
            stock_weights = [item.get_text() for item in stock_weights]

            data = []

            for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
                data.append([name, symbol, weight])

            df3 = pd.DataFrame(data, columns=["holdings", "symbol", "% assets"])
            return df3
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {etf_holdings_url}: {e}")
            print("Retrying...")
            time.sleep(random.uniform(0.0, 3.0))  # Adding a random sleep between retries

    return None  # Return None if scraping fails after retries

# Assuming you have a list of ETF tickers defined as etf_tickers
# etf_tickers = ['EWJ', 'SPY', 'XLK']

etf_data_dict = {}  # Dictionary to store ETF data
not_scraped = []  # List to store tickers that were not successfully scraped

# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the scrape_etf_data function for each ticker
    futures = [executor.submit(scrape_etf_data, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for ticker, future in zip(etf_tickers, as_completed(futures)):
        result = future.result()
        if result is not None:
            etf_data_dict[ticker] = result
        else:
            not_scraped.append(ticker)
        pbar.update(1)

# Create a DataFrame containing data from scraped ETFs
combined_data = pd.concat(etf_data_dict.values(), keys=etf_data_dict.keys(), names=['Ticker'])

# Save the combined DataFrame to a CSV file
# combined_data.to_csv('scraped_etf_data.csv')

if not_scraped:
    print("Tickers that were not successfully scraped:")
    print(not_scraped)


100%|██████████| 104/104 [00:20<00:00,  5.06it/s]


In [10]:
combined_data

holdings    symbol  \
Ticker                                                                   
SPY    0                                    Amazon.com, Inc.      AMZN   
       1                                    Home Depot, Inc.        HD   
       2                                        Walmart Inc.       WMT   
       3                        Costco Wholesale Corporation      COST   
       4                             Lowe\'s Companies, Inc.       LOW   
       5                                   TJX Companies Inc       TJX   
       6                              CVS Health Corporation       CVS   
       7                                McKesson Corporation       MCK   
       8                          O\'Reilly Automotive, Inc.      ORLY   
       9                  JD.com, Inc. Sponsored ADR Class A        JD   
       10                                 Target Corporation       TGT   
       11                                     AutoZone, Inc.       AZO   
       12                            Lululemon Athletica Inc      LULU   
       13                                  Sysco Corporation       SYY   
       14                                  Ross Stores, Inc.      ROST   
QQQ    0                                             Soybean       N/A   
IWM    0                                  Toyota Motor Corp.      7203   
       1                              Sony Group Corporation      6758   
       2                Mitsubishi UFJ Financial Group, Inc.      8306   
       3                                 Keyence Corporation      6861   
       4                                 Tokyo Electron Ltd.      8035   
       5                              Mitsubishi Corporation      8058   
       6                                        Hitachi,Ltd.      6501   
       7               Sumitomo Mitsui Financial Group, Inc.      8316   
       8                           Shin-Etsu Chemical Co Ltd      4063   
       9                     Daiichi Sankyo Company, Limited      4568   
       10                                   Mitsui & Co.,Ltd      8031   
       11                     Takeda Pharmaceutical Co. Ltd.      4502   
       12                               SoftBank Group Corp.      9984   
       13                              Honda Motor Co., Ltd.      7267   
       14                         Recruit Holdings Co., Ltd.      6098   
DIA    0     United States Treasury Notes 1.375% 15-NOV-2031       N/A   
       1     United States Treasury Notes 3.125% 15-NOV-2028       N/A   
       2     United States Treasury Notes 2.625% 15-FEB-2029       N/A   
       3      United States Treasury Bond 1.875% 15-FEB-2051       N/A   
       4     United States Treasury Notes 2.375% 15-AUG-2024       N/A   
       5       United States Treasury Bond 3.75% 15-NOV-2043       N/A   
       6      United States Treasury Notes 2.75% 15-FEB-2028       N/A   
       7     United States Treasury Notes 1.625% 15-MAY-2026       N/A   
       8       United States Treasury Notes 2.0% 15-FEB-2025       N/A   
       9      United States Treasury Notes 2.25% 15-FEB-2027       N/A   
       10    United States Treasury Notes 0.375% 30-NOV-2025       N/A   
       11     United States Treasury Notes 0.25% 15-MAY-2024       N/A   
       12    United States Treasury Notes 0.375% 15-AUG-2024       N/A   
       13     United States Treasury Notes 0.75% 31-AUG-2026       N/A   
       14      United States Treasury Notes 1.5% 15-FEB-2025       N/A   
IYT    0                           Union Pacific Corporation       UNP   
       1                 United Parcel Service, Inc. Class B       UPS   
       2                             Uber Technologies, Inc.      UBER   
       3                     Old Dominion Freight Line, Inc.      ODFL   
       4                                   FedEx Corporation       FDX   
       5                               Delta Air Lines, Inc.       DAL   
       6                        Norfolk Southern Corpora

In [11]:
combined_data.T

Ticker                 SPY                                  \
                        0                 1             2    
holdings  Amazon.com, Inc.  Home Depot, Inc.  Walmart Inc.   
symbol                AMZN                HD           WMT   
% assets            21.09%             9.25%         8.41%   

Ticker                                                           \
                                    3                        4    
holdings  Costco Wholesale Corporation  Lowe\'s Companies, Inc.   
symbol                            COST                      LOW   
% assets                         7.49%                    4.72%   

Ticker                                                                     \
                         5                       6                     7    
holdings  TJX Companies Inc  CVS Health Corporation  McKesson Corporation   
symbol                  TJX                     CVS                   MCK   
% assets              4.61%                   4.36%                 3.96%   

Ticker                                                                    \
                                  8                                   9    
holdings  O\'Reilly Automotive, Inc.  JD.com, Inc. Sponsored ADR Class A   
symbol                          ORLY                                  JD   
% assets                       3.95%                               3.85%   

Ticker                                                                 \
                          10              11                       12   
holdings  Target Corporation  AutoZone, Inc.  Lululemon Athletica Inc   
symbol                   TGT             AZO                     LULU   
% assets               3.46%           2.60%                    2.48%   

Ticker                                              QQQ                 IWM  \
                         13                 14       0                   0    
holdings  Sysco Corporation  Ross Stores, Inc.  Soybean  Toyota Motor Corp.   
symbol                  SYY               ROST      N/A                7203   
% assets              2.42%              2.21%  100.00%               5.10%   

Ticker                                                                  \
                              1                                     2    
holdings  Sony Group Corporation  Mitsubishi UFJ Financial Group, Inc.   
symbol                      6758                                  8306   
% assets                   3.10%                                 2.51%   

Ticker                                                                      \
                           3                    4                       5    
holdings  Keyence Corporation  Tokyo Electron Ltd.  Mitsubishi Corporation   
symbol                   6861                 8035                    8058   
% assets                2.32%                1.85%                   1.76%   

Ticker                                                         \
                    6                                      7    
holdings  Hitachi,Ltd.  Sumitomo Mitsui Financial Group, Inc.   
symbol            6501                                   8316   
% assets         1.71%                                  1.66%   

Ticker                                                                \
                                 8                                9    
holdings  Shin-Etsu Chemical Co Ltd  Daiichi Sankyo Company, Limited   
symbol                         4063                             4568   
% assets                      1.66%                            1.60%   

Ticker                                                      \
                        10                              11   
holdings  Mitsui & Co.,Ltd  Takeda Pharmaceutical Co. Ltd.   
symbol                8031                            4502   
% assets             1.47%                           1.41%   

Ticker                                                 \
       

# workable one that downloads correctly

In [19]:
etf_tickers = pd.read_csv('etf_tickers.csv').dropna()
etf_tickers = etf_tickers['ticker'].values.tolist()
etf_tickers

['SPY',
 'QQQ',
 'IWM',
 'DIA',
 'IYT',
 'SMH',
 'EWJ',
 'BBJP',
 'XLK',
 'XLE',
 'XLY',
 'XLI',
 'XLF',
 'XLC',
 'XLV',
 'XLP',
 'XLU',
 'XLB',
 'XLRE',
 'XPH',
 'XBI',
 'XRT',
 'XHB',
 'RTH',
 'IYC',
 'ITB',
 'EWT',
 'SKYY',
 'CLOU',
 'ROBO',
 'CHIQ',
 'KWEB',
 'DBA',
 'DBB',
 'USO',
 'UNG',
 'UGA',
 'WEAT',
 'CORN',
 'GLD',
 'SOYB',
 'CANE',
 'GLDM',
 'SLV',
 'GDX',
 'GDXJ',
 'NUGT',
 'JNUG',
 'DUST',
 'JDST',
 'PPLT',
 'COPX',
 'UUP',
 'FXY',
 'FXE',
 'FXB',
 'FXC',
 'VXX',
 'UVXY',
 'SVXY',
 'UVIX',
 'SVIX',
 'BITO',
 'BND',
 'AGG',
 'BNDX',
 'LQD',
 'HYG',
 'JNK',
 'TIP',
 'VTIP',
 'GOVT',
 'EMB',
 'IGSB',
 'IEF',
 'TLT',
 'IGOV',
 'BWX',
 'FLIA',
 'BWZ',
 'ISHG',
 'TYO',
 'TBF',
 'TMV',
 'TBT',
 'TYD',
 'UBT',
 'TMF',
 'TQQQ',
 'SQQQ',
 'SH',
 'SPXU',
 'SPXS',
 'SOXL',
 'SOXS',
 'TNA',
 'TZA',
 'FNGD',
 'YINN',
 'YANG',
 'BOIL',
 'UCO',
 'SCO',
 'AGQ']

In [20]:
len(etf_tickers)

104

In [16]:
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

def scrape_etf_data(ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings' 
    response = requests.get(etf_holdings_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    etf_name = etf_holdings_url.split('/')[-2]

    rows = soup.find_all('tr', {'class': 'holding'})

    stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
    stock_names = soup.find_all('td', {'data-th': 'Holding'})
    stock_weights = soup.find_all('td', {'data-th': '% Assets'})

    stock_symbols = [item.get_text() for item in stock_symbols]
    stock_names = [item.get_text() for item in stock_names]
    stock_weights = [item.get_text() for item in stock_weights]

    data = []

    for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
        data.append([name, symbol, weight])

    df = pd.DataFrame(data, columns=["holdings", "symbol", "% assets"])
    return df

etf_tickers = ['SPY', 'EWJ', 'XLK']  # Add your list of ETF tickers here
etf_data_dict = {}  # Dictionary to store ETF data

# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Submit the scrape_etf_data function for each ticker
    futures = [executor.submit(scrape_etf_data, ticker) for ticker in etf_tickers]
    # Retrieve the results as they complete
    for ticker, future in zip(etf_tickers, as_completed(futures)):
        result = future.result()
        if result is not None:
            etf_data_dict[ticker] = result
        pbar.update(1)

        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(2, 5)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Store the ETF data in a database or file
for ticker, df in etf_data_dict.items():
    df.to_csv(f'{ticker}_holdings.csv', index=False)

# Display the stored data for a specific ETF
selected_etf = 'SPY'
if selected_etf in etf_data_dict:
    print(f"Holdings data for {selected_etf}:")
    print(etf_data_dict[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

100%|██████████| 3/3 [00:12<00:00,  4.27s/it]

Holdings data for SPY:
                           holdings symbol % assets
0                        Apple Inc.   AAPL    7.19%
1             Microsoft Corporation   MSFT    6.51%
2                  Amazon.com, Inc.   AMZN    3.33%
3                NVIDIA Corporation   NVDA    2.95%
4             Alphabet Inc. Class A  GOOGL    2.03%
5       Meta Platforms Inc. Class A   META    1.84%
6                       Tesla, Inc.   TSLA    1.83%
7             Alphabet Inc. Class C   GOOG    1.76%
8   Berkshire Hathaway Inc. Class B  BRK.B    1.67%
9   UnitedHealth Group Incorporated    UNH    1.25%
10             JPMorgan Chase & Co.    JPM    1.22%
11                Johnson & Johnson    JNJ    1.17%
12          Exxon Mobil Corporation    XOM    1.16%
13                Visa Inc. Class A      V    1.03%
14                    Broadcom Inc.   AVGO    0.98%


# Download top 15 holdings, save into 1 df

In [72]:
etf_small = pd.read_csv('etf_small.csv').dropna()
etf_small = etf_small['ticker'].values.tolist()
etf_small

['AGG',
 'BBJP',
 'BND',
 'BNDX',
 'BWX',
 'BWZ',
 'CHIQ',
 'CLOU',
 'COPX',
 'DBA',
 'DBB',
 'DIA',
 'EMB',
 'EWJ',
 'EWT',
 'FLIA',
 'FXI',
 'GDX',
 'GDXJ',
 'GLD',
 'GLDM',
 'GOVT',
 'HYG',
 'IEF',
 'IEMG',
 'IGOV',
 'IGSB',
 'ISHG',
 'ITB',
 'IVV',
 'IWM',
 'IYC',
 'IYT',
 'JNK',
 'KWEB',
 'LQD',
 'MCHI',
 'PPLT',
 'QQQ',
 'ROBO',
 'RTH',
 'SKYY',
 'SLV',
 'SMH',
 'SPY',
 'TIP',
 'TLT',
 'VOO',
 'VTIP',
 'XBI',
 'XHB',
 'XLB',
 'XLC',
 'XLE',
 'XLF',
 'XLI',
 'XLK',
 'XLP',
 'XLRE',
 'XLU',
 'XLV',
 'XLY',
 'XPH',
 'XRT']

In [76]:
len(etf_small)

64

In [77]:
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

def scrape_etf_data(ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings' 
    response = requests.get(etf_holdings_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    etf_name = etf_holdings_url.split('/')[-2]

    rows = soup.find_all('tr', {'class': 'holding'})

    stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
    stock_names = soup.find_all('td', {'data-th': 'Holding'})
    stock_weights = soup.find_all('td', {'data-th': '% Assets'})

    stock_symbols = [item.get_text() for item in stock_symbols]
    stock_names = [item.get_text() for item in stock_names]
    stock_weights = [item.get_text() for item in stock_weights]

    data = []

    for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
        data.append([etf_name, name, symbol, weight])

    df = pd.DataFrame(data, columns=["ETF", "holdings", "symbol", "% assets"])
    return df


# List of ETF tickers in the desired order
etf_tickers = etf_small#['BBJP', 'BND', 'AGG']  # Add more tickers as needed

# Dictionary to store ETF data
etf_data_dict = {}

# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Loop through each ticker in the desired order
    for ticker in etf_tickers:
        future = executor.submit(scrape_etf_data, ticker)
        result = future.result()
        if result is not None:
            etf_data_dict[ticker] = result
        else:
            not_scraped.append(ticker)
        pbar.update(1)
        
        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0.0, 3.0)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Display the stored data for each ETF
for ticker, df in etf_data_dict.items():
    print(f"Holdings data for {ticker}:")
    print(df)

100%|██████████| 64/64 [04:18<00:00,  4.05s/it]

Holdings data for AGG:
    ETF                                           holdings symbol % assets
0   AGG           BlackRock Funds III Instl Money Mkt Fund  Other    7.41%
1   AGG  GNMA II TBA 30yr 2% February Delivery 2.000% 0...    N/A    0.88%
2   AGG    United States Treasury Notes 0.375% 31-JAN-2026    N/A    0.79%
3   AGG    United States Treasury Notes 0.125% 31-JUL-2023    N/A    0.73%
4   AGG     United States Treasury Bond 1.875% 15-FEB-2051    N/A    0.65%
5   AGG  UMBS TBA 30yr 2.5% March Delivery 2.500% 01-OC...    N/A    0.63%
6   AGG     United States Treasury Notes 0.25% 30-SEP-2023    N/A    0.59%
7   AGG  UMBS TBA 15yr 1.5% February Delivery 1.500% 01...    N/A    0.58%
8   AGG       United States Treasury Bond 2.0% 15-AUG-2051    N/A    0.55%
9   AGG  UMBS TBA 30yr 2.5% February Delivery 2.500% 01...    N/A    0.52%
10  AGG      United States Treasury Notes 1.5% 15-AUG-2026    N/A    0.52%
11  AGG  FNMA 30yr Pool#MA4437 2.000% 01-Oct-2051 2.0% ...    N/A    0.51%
12

In [86]:
# Create an empty DataFrame to store the combined data
combined_dataframe = pd.DataFrame()

# Iterate through each ETF's DataFrame and concatenate them
for ticker, df in etf_data_dict.items():
    df['ETF'] = ticker  # Add a column for ETF ticker
    combined_dataframe = pd.concat([combined_dataframe, df], ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:")
combined_dataframe

Combined DataFrame:


,ETF,holdings,symbol,% assets
0,AGG,BlackRock Funds III Instl Money Mkt Fund,Other,7.41%
1,AGG,GNMA II TBA 30yr 2% February Delivery 2.000% 0...,N/A,0.88%
2,AGG,United States Treasury Notes 0.375% 31-JAN-2026,N/A,0.79%
3,AGG,United States Treasury Notes 0.125% 31-JUL-2023,N/A,0.73%
4,AGG,United States Treasury Bond 1.875% 15-FEB-2051,N/A,0.65%
5,AGG,UMBS TBA 30yr 2.5% March Delivery 2.500% 01-OC...,N/A,0.63%
6,AGG,United States Treasury Notes 0.25% 30-SEP-2023,N/A,0.59%
7,AGG,UMBS TBA 15yr 1.5% February Delivery 1.500% 01...,N/A,0.58%
8,AGG,United States Treasury Bond 2.0% 15-AUG-2051,N/A,0.55%
9,AGG,UMBS TBA 30yr 2.5% February Delivery 2.500% 01...,N/A,0.52%


In [87]:
combined_dataframe.to_csv('etf_small_bw.csv')

In [90]:
# Load your data into a DataFrame (assuming the data is in a CSV file)
combined_dataframe = pd.read_csv('etf_small_bw.csv')

# Group the data by ETF
grouped_data = combined_dataframe.groupby('ETF')

# Iterate through each ETF group and display the data
for etf, group in grouped_data:
    print(f"Holdings data for {etf}:")
    print(group)
    print('\n')

Holdings data for AGG:
    ETF                                           holdings symbol % assets
0   AGG           BlackRock Funds III Instl Money Mkt Fund  Other    7.41%
1   AGG  GNMA II TBA 30yr 2% February Delivery 2.000% 0...    NaN    0.88%
2   AGG    United States Treasury Notes 0.375% 31-JAN-2026    NaN    0.79%
3   AGG    United States Treasury Notes 0.125% 31-JUL-2023    NaN    0.73%
4   AGG     United States Treasury Bond 1.875% 15-FEB-2051    NaN    0.65%
5   AGG  UMBS TBA 30yr 2.5% March Delivery 2.500% 01-OC...    NaN    0.63%
6   AGG     United States Treasury Notes 0.25% 30-SEP-2023    NaN    0.59%
7   AGG  UMBS TBA 15yr 1.5% February Delivery 1.500% 01...    NaN    0.58%
8   AGG       United States Treasury Bond 2.0% 15-AUG-2051    NaN    0.55%
9   AGG  UMBS TBA 30yr 2.5% February Delivery 2.500% 01...    NaN    0.52%
10  AGG      United States Treasury Notes 1.5% 15-AUG-2026    NaN    0.52%
11  AGG  FNMA 30yr Pool#MA4437 2.000% 01-Oct-2051 2.0% ...    NaN    0.51%
12

In [5]:
import pandas as pd 

# List of CSV files
csv_files = [
    'AGQ_holdings.csv', 'BBJP_holdings.csv', 'BITO_holdings.csv', 'BND_holdings.csv', 'BNDX_holdings.csv',
    'BOIL_holdings.csv', 'BWZ_holdings.csv', 'BWX_holdings.csv', 'CANE_holdings.csv', 'CHIQ_holdings.csv',
    'CLOU_holdings.csv', 'COPX_holdings.csv', 'CORN_holdings.csv', 'DBA_holdings.csv', 'DBB_holdings.csv',
    'DIA_holdings.csv', 'DUST_holdings.csv', 'EWT_holdings.csv', 'EWJ_holdings.csv', 'FLIA_holdings.csv',
    'FNGD_holdings.csv', 'FXB_holdings.csv', 'FXC_holdings.csv', 'FXE_holdings.csv', 'FXY_holdings.csv',
    'GDX_holdings.csv', 'GDXJ_holdings.csv', 'GLD_holdings.csv', 'GLDM_holdings.csv', 'GOVT_holdings.csv',
    'HYG_holdings.csv', 'IEF_holdings.csv', 'IGOV_holdings.csv', 'IGSB_holdings.csv', 'ISHG_holdings.csv',
    'ITB_holdings.csv',  'IWM_holdings.csv', 'IYC_holdings.csv',  'JDST_holdings.csv',
    'JNK_holdings.csv', 'JNUG_holdings.csv', 'KWEB_holdings.csv', 'LQD_holdings.csv', 'NUGT_holdings.csv',
    'PPLT_holdings.csv', 'QQQ_holdings.csv', 'RTH_holdings.csv', 'SCO_holdings.csv', 'SH_holdings.csv',
    'SKYY_holdings.csv', 'SLV_holdings.csv', 'SMH_holdings.csv', 'SOXL_holdings.csv', 'SOXS_holdings.csv',
    'SOYB_holdings.csv', 'SPXU_holdings.csv', 'SPXS_holdings.csv', 'SPY_holdings.csv', 'SQQQ_holdings.csv',
    'SVIX_holdings.csv', 'SVXY_holdings.csv', 'TBF_holdings.csv', 'TBT_holdings.csv', 'TIP_holdings.csv',
    'TLT_holdings.csv', 'TMF_holdings.csv', 'TMV_holdings.csv', 'TNA_holdings.csv', 'TQQQ_holdings.csv',
    'TZA_holdings.csv', 'UBT_holdings.csv', 'UCO_holdings.csv', 'UGA_holdings.csv', 'UNG_holdings.csv',
    'USO_holdings.csv', 'UUP_holdings.csv', 'UVIX_holdings.csv', 'UVXY_holdings.csv', 'VTIP_holdings.csv',
    'VXX_holdings.csv', 'WEAT_holdings.csv', 'XBI_holdings.csv', 'XHB_holdings.csv', 'XLB_holdings.csv',
    'XLC_holdings.csv', 'XLE_holdings.csv', 'XLF_holdings.csv', 'XLI_holdings.csv', 'XLK_holdings.csv',
    'XLP_holdings.csv', 'XLRE_holdings.csv', 'XLU_holdings.csv', 'XLV_holdings.csv', 'XLY_holdings.csv',
    'XPH_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv',
    'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv', 'XRT_holdings.csv',
    'YINN_holdings.csv', 'YANG_holdings.csv'
]

  # Add your CSV files

# Create a Pandas Excel writer using XlsxWriter as the engine
excel_writer = pd.ExcelWriter('combined_holdings.xlsx', engine='xlsxwriter')

# Loop through each CSV file and write its contents to a new sheet
for csv_file in csv_files:
    # Read CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Get the ETF name from the CSV filename
    etf_name = csv_file.split('_')[0]
    
    # Write the DataFrame to a new sheet with the ETF name
    df.to_excel(excel_writer, sheet_name=etf_name, index=False)

# Save the Excel workbook
excel_writer.save()

print('Merged CSV files into a single Excel file.')


C:\Users\Jonat\AppData\Local\Temp\ipykernel_36820\1909152908.py:46: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


Merged CSV files into a single Excel file.


In [99]:
# Convert float values to strings (assuming the column names are correct)
combined_dataframe['symbol'] = combined_dataframe['symbol'].astype(str)
combined_dataframe['% assets'] = combined_dataframe['% assets'].astype(str)

# Define the aggregation functions for each column
agg_functions = {
    'holdings': 'first',    # Use 'first' to get the first value (ETF name)
    'symbol': ', '.join,    # Concatenate symbols for the same ETF
    '% assets': ', '.join   # Concatenate % assets for the same ETF
}

# Group the data by ETF and apply aggregation
aggregated_data = combined_dataframe.groupby('ETF').agg(agg_functions).reset_index()

# Print the aggregated data
aggregated_data.iloc[0]

ETF                                                       AGG
holdings             BlackRock Funds III Instl Money Mkt Fund
symbol      Other, nan, nan, nan, nan, nan, nan, nan, nan,...
% assets    7.41%, 0.88%, 0.79%, 0.73%, 0.65%, 0.63%, 0.59...
Name: 0, dtype: object

In [102]:
aggregated_data.loc[1, 'holdings']

'Toyota Motor Corp.'

In [21]:
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd

def scrape_etf_data(ticker):
    etf_holdings_url = f'https://etfdb.com/etf/{ticker}/#holdings' 
    response = requests.get(etf_holdings_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    etf_name = etf_holdings_url.split('/')[-2]

    rows = soup.find_all('tr', {'class': 'holding'})

    stock_symbols = soup.find_all('td', {'data-th': 'Symbol'})
    stock_names = soup.find_all('td', {'data-th': 'Holding'})
    stock_weights = soup.find_all('td', {'data-th': '% Assets'})

    stock_symbols = [item.get_text() for item in stock_symbols]
    stock_names = [item.get_text() for item in stock_names]
    stock_weights = [item.get_text() for item in stock_weights]

    data = []

    for name, symbol, weight in zip(stock_names, stock_symbols, stock_weights):
        data.append([name, symbol, weight])

    df = pd.DataFrame(data, columns=["holdings", "symbol", "% assets"])
    return df


# List of ETF tickers in the desired order
etf_tickers = etf_tickers#['BBJP', 'BND', 'AGG']  # Add more tickers as needed

# Dictionary to store ETF data
etf_data_dict = {}

# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers)) as executor, tqdm(total=len(etf_tickers)) as pbar:
    # Loop through each ticker in the desired order
    for ticker in etf_tickers:
        future = executor.submit(scrape_etf_data, ticker)
        result = future.result()
        if result is not None:
            etf_data_dict[ticker] = result
        else:
            not_scraped.append(ticker)
        pbar.update(1)
        
        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0.0, 3.0)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Display the stored data for each ETF
for ticker, df in etf_data_dict.items():
    print(f"Holdings data for {ticker}:")
    print(df)


100%|██████████| 104/104 [07:10<00:00,  4.14s/it]

Holdings data for SPY:
                           holdings symbol % assets
0                        Apple Inc.   AAPL    7.19%
1             Microsoft Corporation   MSFT    6.51%
2                  Amazon.com, Inc.   AMZN    3.33%
3                NVIDIA Corporation   NVDA    2.95%
4             Alphabet Inc. Class A  GOOGL    2.03%
5       Meta Platforms Inc. Class A   META    1.84%
6                       Tesla, Inc.   TSLA    1.83%
7             Alphabet Inc. Class C   GOOG    1.76%
8   Berkshire Hathaway Inc. Class B  BRK.B    1.67%
9   UnitedHealth Group Incorporated    UNH    1.25%
10             JPMorgan Chase & Co.    JPM    1.22%
11                Johnson & Johnson    JNJ    1.17%
12          Exxon Mobil Corporation    XOM    1.16%
13                Visa Inc. Class A      V    1.03%
14                    Broadcom Inc.   AVGO    0.98%
Holdings data for QQQ:
                        holdings symbol % assets
0                     Apple Inc.   AAPL   10.97%
1          Microsoft Cor

# Display the stored data for a specific ETF

In [33]:
# Display the stored data for a specific ETF
selected_etf = 'EWJ'
if selected_etf in etf_data_dict:
    print(f"Holdings data for {selected_etf}:")
    print(etf_data_dict[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

Holdings data for EWJ:
                                 holdings symbol % assets
0                      Toyota Motor Corp.   7203    5.10%
1                  Sony Group Corporation   6758    3.10%
2    Mitsubishi UFJ Financial Group, Inc.   8306    2.51%
3                     Keyence Corporation   6861    2.32%
4                     Tokyo Electron Ltd.   8035    1.85%
5                  Mitsubishi Corporation   8058    1.76%
6                            Hitachi,Ltd.   6501    1.71%
7   Sumitomo Mitsui Financial Group, Inc.   8316    1.66%
8               Shin-Etsu Chemical Co Ltd   4063    1.66%
9         Daiichi Sankyo Company, Limited   4568    1.60%
10                       Mitsui & Co.,Ltd   8031    1.47%
11         Takeda Pharmaceutical Co. Ltd.   4502    1.41%
12                   SoftBank Group Corp.   9984    1.40%
13                  Honda Motor Co., Ltd.   7267    1.40%
14             Recruit Holdings Co., Ltd.   6098    1.38%


In [35]:
type(etf_data_dict)#.to_csv('etf_holdings.csv')

dict

In [37]:
etf_data_dict.items()

dict_items([('SPY',                            holdings symbol % assets
0                        Apple Inc.   AAPL    7.19%
1             Microsoft Corporation   MSFT    6.51%
2                  Amazon.com, Inc.   AMZN    3.33%
3                NVIDIA Corporation   NVDA    2.95%
4             Alphabet Inc. Class A  GOOGL    2.03%
5       Meta Platforms Inc. Class A   META    1.84%
6                       Tesla, Inc.   TSLA    1.83%
7             Alphabet Inc. Class C   GOOG    1.76%
8   Berkshire Hathaway Inc. Class B  BRK.B    1.67%
9   UnitedHealth Group Incorporated    UNH    1.25%
10             JPMorgan Chase & Co.    JPM    1.22%
11                Johnson & Johnson    JNJ    1.17%
12          Exxon Mobil Corporation    XOM    1.16%
13                Visa Inc. Class A      V    1.03%
14                    Broadcom Inc.   AVGO    0.98%), ('QQQ',                         holdings symbol % assets
0                     Apple Inc.   AAPL   10.97%
1          Microsoft Corporation   MSFT 

In [ ]:
THISO NE

In [38]:
# Loop through each ETF in the dictionary
for ticker, data in etf_data_dict.items():
    csv_file = f'{ticker}_holdings.csv'
    fieldnames = list(data.keys())  # Assuming the keys are the same for all data
    with open(csv_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(data['holdings'])):
            row = {key: data[key][i] for key in fieldnames}
            writer.writerow(row)

print('Data has been written to CSV files.')

Data has been written to CSV files.


# IVV, FXI, MCHI, IEMG holdings

In [67]:
# List of ETF tickers in the desired order
#etf_tickers_snp = ['SPY', 'IVV', 'VOO']  # Add more tickers as needed   ###############
etf_tickers_bw = ['IVV', 'FXI', 'MCHI', 'IEMG']  # Add more tickers as needed   ###############

# Dictionary to store ETF data
etf_tickers_bw_dict = {}   ###############

# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers_bw)) as executor, tqdm(total=len(etf_tickers_bw)) as pbar:   ###############
#with ThreadPoolExecutor(max_workers=10) as executor, tqdm(total=len(etf_tickers_snp)) as pbar:   ###############
    # Loop through each ticker in the desired order
    for ticker in etf_tickers_bw:   ###############
        future = executor.submit(scrape_etf_data, ticker)
        result = future.result()
        if result is not None:
            etf_tickers_bw_dict[ticker] = result ###############
        else:
            not_scraped.append(ticker)
        pbar.update(1)
        
        # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0.0, 3.0)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

# Display the stored data for each ETF
for ticker, etf_tickers_bw in etf_tickers_bw_dict.items():  ###############
    print(f"Holdings data for {ticker}:")
    print(etf_tickers_bw)

100%|██████████| 3/3 [00:10<00:00,  3.37s/it]

Holdings data for SPY:
                           holdings symbol % assets
0                        Apple Inc.   AAPL    7.19%
1             Microsoft Corporation   MSFT    6.51%
2                  Amazon.com, Inc.   AMZN    3.33%
3                NVIDIA Corporation   NVDA    2.95%
4             Alphabet Inc. Class A  GOOGL    2.03%
5       Meta Platforms Inc. Class A   META    1.84%
6                       Tesla, Inc.   TSLA    1.83%
7             Alphabet Inc. Class C   GOOG    1.76%
8   Berkshire Hathaway Inc. Class B  BRK.B    1.67%
9   UnitedHealth Group Incorporated    UNH    1.25%
10             JPMorgan Chase & Co.    JPM    1.22%
11                Johnson & Johnson    JNJ    1.17%
12          Exxon Mobil Corporation    XOM    1.16%
13                Visa Inc. Class A      V    1.03%
14                    Broadcom Inc.   AVGO    0.98%
Holdings data for IVV:
                           holdings symbol % assets
0                        Apple Inc.   AAPL    7.18%
1             Micr

In [45]:
# # Display the stored data for a specific ETF
# selected_etf = 'IVV'
# if selected_etf in etf_data_dict_ivv:
#     print(f"Holdings data for {selected_etf}:")
#     print(etf_data_dict_ivv[selected_etf])
# else:
#     print(f"No data found for {selected_etf}.")
    
    
    # Display the stored data for a specific ETF
selected_etf = 'SPY'
if selected_etf in etf_data_dict:
    print(f"Holdings data for {selected_etf}:")
    print(etf_data_dict[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

Holdings data for SPY:
                           holdings symbol % assets
0                        Apple Inc.   AAPL    7.19%
1             Microsoft Corporation   MSFT    6.51%
2                  Amazon.com, Inc.   AMZN    3.33%
3                NVIDIA Corporation   NVDA    2.95%
4             Alphabet Inc. Class A  GOOGL    2.03%
5       Meta Platforms Inc. Class A   META    1.84%
6                       Tesla, Inc.   TSLA    1.83%
7             Alphabet Inc. Class C   GOOG    1.76%
8   Berkshire Hathaway Inc. Class B  BRK.B    1.67%
9   UnitedHealth Group Incorporated    UNH    1.25%
10             JPMorgan Chase & Co.    JPM    1.22%
11                Johnson & Johnson    JNJ    1.17%
12          Exxon Mobil Corporation    XOM    1.16%
13                Visa Inc. Class A      V    1.03%
14                    Broadcom Inc.   AVGO    0.98%


# IVV, FXI, MCHI, IEMG INFO

In [69]:
#results = []
etf_tickers_snp = ['SPY', 'IVV', 'VOO'] 
# Create a ThreadPoolExecutor with max_workers equal to the number of tickers
with ThreadPoolExecutor(max_workers=len(etf_tickers_snp)) as executor, tqdm(total=len(etf_tickers_snp)) as pbar:
    # Submit the get_etf_info function for each ticker
    futures = [executor.submit(get_etf_info, ticker) for ticker in etf_tickers_snp]
    # Retrieve the results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            results.append(result)
        pbar.update(1)
        
     # Introduce a random sleep interval between requests
        sleep_interval = random.uniform(0, 3)  # Random sleep between 2 to 5 seconds
        time.sleep(sleep_interval)

        
        
# Convert the list of dictionaries to a DataFrame
df_snp = pd.DataFrame(results)


print("Data saved to database.")

# Display the DataFrame
df_snp

100%|██████████| 3/3 [00:10<00:00,  3.50s/it]

Data saved to database.


,ETF Ticker,Issuer,Brand,ETF Home Page,Category,Asset Class Size,Weighting,Expense Ratio,PE Ratio,Peer PE Ratio,ETF Dividend,ETF Div Yield,ETF Peer Dividend,ETF Peer Div Yield,1 Month Return,3 Month Return,YTD Return,1 Year Return,3 Year Return,5 Year Return,Peer 1 Month Return,Peer 3 Month Return,Peer YTD Return,Peer 1 Year Return,Peer 3 Year Return,Peer 5 Year Return,AUM,Shares float,1 Month Avg. Volume,3 Month Avg. Volume
0,IEMG,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Emerging Markets Equities,Large-Cap,Market Cap,0.09%,10.56,7.12,$ 0.48,2.28%,$ 1.14,2.28%,1.85%,4.03%,8.26%,3.15%,0.97%,1.46%,2.44%,4.06%,8.41%,5.32%,3.30%,0.97%,"$72,635.9 M","1,450.2 M","11,761,079","10,662,180"
1,MCHI,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,China Equities,Large-Cap,Market Cap,0.58%,10.89,9.73,$ 0.43,2.42%,$ 1.14,2.42%,5.12%,0.91%,0.34%,-1.17%,-11.89%,-4.06%,4.06%,-0.96%,-1.31%,-6.70%,-5.26%,0.05%,"$7,957.2 M",169.2 M,"4,310,742","3,990,070"
2,FXI,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,China Equities,Large-Cap,Market Cap,0.74%,10.62,9.73,$ 0.15,2.61%,$ 0.75,2.61%,6.07%,1.92%,2.43%,0.24%,-10.01%,-5.79%,4.06%,-0.96%,-1.31%,-6.70%,-5.26%,0.05%,"$5,370.9 M",187.4 M,"31,644,312","30,885,116"
3,IVV,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Large Cap Growth Equities,Large-Cap,Market Cap,0.03%,19.66,15.24,$ 1.34,1.48%,$ 6.62,1.48%,1.70%,8.93%,17.54%,7.99%,11.77%,11.33%,1.57%,9.07%,18.87%,6.52%,7.10%,6.04%,"$347,092.0 M",774.7 M,"4,146,967","4,004,722"
4,IVV,BlackRock Financial Management,iShares,http://us.ishares.com/product_info/fund/overvi...,Large Cap Growth Equities,Large-Cap,Market Cap,0.03%,19.66,15.24,$ 1.34,1.48%,$ 6.62,1.48%,1.70%,8.93%,17.54%,7.99%,11.77%,11.33%,1.57%,9.07%,18.87%,6.52%,7.10%,6.04%,"$347,092.0 M",774.7 M,"4,146,967","4,004,722"
5,SPY,State Street,SPDR,https://www.spdrs.com/product/fund.seam?ticker...,Large Cap Growth Equities,Large-Cap,Market Cap,0.09%,17.86,15.24,$ 1.64,1.46%,$ 6.52,1.46%,1.68%,8.91%,17.48%,7.90%,11.72%,11.22%,1.57%,9.07%,18.87%,6.52%,7.10%,6.04%,"$418,862.0 M",939.8 M,"72,177,816","77,834,752"
6,VOO,Vanguard,Vanguard,https://advisors.vanguard.com/investments/prod...,Large Cap Growth Equities,Large-Cap,Market Cap,0.03%,20.10,15.24,$ 1.58,1.52%,$ 6.20,1.52%,1.71%,8.97%,17.58%,7.99%,11.79%,11.29%,1.57%,9.07%,18.87%,6.52%,7.10%,6.04%,"$334,782.0 M",811.9 M,"3,746,079","3,784,278"


In [71]:
df_snp.to_csv('ivv_spy_voo.csv')

import csv
# Specify the CSV file name
csv_file = 'etf_holdings.csv'

data_dict = etf_data_dict

# Get the keys for the fieldnames (assuming the keys are the same for all ETFs)
fieldnames = list(etf_data_dict[next(iter(etf_data_dict))].keys())

# Write the dictionary data to a CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for etf, data in etf_data_dict.items():
        for i in range(len(data['holdings'])):
            row = {key: data[key][i] for key in fieldnames}
            writer.writerow(row)

print(f'Data has been written to {csv_file}')

In [74]:
# Display the stored data for a specific ETF
selected_etf = 'BBJP'
if selected_etf in etf_data_dict:
    print(f"Holdings data for {selected_etf}:")
    print(etf_data_dict[selected_etf])
else:
    print(f"No data found for {selected_etf}.")

,holdings,symbol,% assets
0,UnitedHealth Group Incorporated,UNH,9.24%
1,Johnson & Johnson,JNJ,8.89%
2,Eli Lilly and Company,LLY,8.19%
3,"Merck & Co., Inc.",MRK,5.32%
4,"AbbVie, Inc.",ABBV,5.26%
5,Thermo Fisher Scientific Inc.,TMO,4.17%
6,Pfizer Inc.,PFE,4.00%
7,Abbott Laboratories,ABT,3.62%
8,Danaher Corporation,DHR,3.36%
9,Amgen Inc.,AMGN,2.74%


# GOOD

In [26]:
combined_df

,holdings,symbol,% assets,Ticker
0,Toyota Motor Corp.,7203,5.10%,SPY
1,Sony Group Corporation,6758,3.10%,SPY
2,"Mitsubishi UFJ Financial Group, Inc.",8306,2.51%,SPY
3,Keyence Corporation,6861,2.32%,SPY
4,Tokyo Electron Ltd.,8035,1.85%,SPY
5,Mitsubishi Corporation,8058,1.76%,SPY
6,"Hitachi,Ltd.",6501,1.71%,SPY
7,"Sumitomo Mitsui Financial Group, Inc.",8316,1.66%,SPY
8,Shin-Etsu Chemical Co Ltd,4063,1.66%,SPY
9,"Daiichi Sankyo Company, Limited",4568,1.60%,SPY


In [31]:
combined_df.to_csv('combined_df.csv')

In [39]:
combined_df

,holdings,symbol,% assets,Ticker
0,Toyota Motor Corp.,7203,5.10%,SPY
1,Sony Group Corporation,6758,3.10%,SPY
2,"Mitsubishi UFJ Financial Group, Inc.",8306,2.51%,SPY
3,Keyence Corporation,6861,2.32%,SPY
4,Tokyo Electron Ltd.,8035,1.85%,SPY
5,Mitsubishi Corporation,8058,1.76%,SPY
6,"Hitachi,Ltd.",6501,1.71%,SPY
7,"Sumitomo Mitsui Financial Group, Inc.",8316,1.66%,SPY
8,Shin-Etsu Chemical Co Ltd,4063,1.66%,SPY
9,"Daiichi Sankyo Company, Limited",4568,1.60%,SPY


In [43]:
etf_tickers = combined_df['Ticker'].unique()
etf_tickers

array(['SPY', 'EWJ', 'XLK'], dtype=object)

# Save the DataFrame to an SQLite3 database
db_path = 'etf_holdings.db'
conn = sqlite3.connect(db_path)
df.to_sql('etf_data', conn, if_exists='replace', index=False)
conn.close()

NameError: name 'etf_holdings_dict' is not defined

In [44]:
df1.T.to_csv('etf_info.csv', index=True)

# Raw testing code

In [11]:
etf_url = 'https://etfdb.com/etf/EWJ/#etf-ticker-profile'
etf_ticker = etf_url.split('/')[-2]
response = requests.get(etf_url)
soup = BeautifulSoup(response.text, 'html.parser')


#Basic Information
issuer = soup.find('span', text='Issuer').find_next_sibling('span').get_text()
brand = soup.find('span', text='Brand').find_next_sibling('span').get_text()
etf_home_page = soup.find('span', text='ETF Home Page').find_next_sibling('span').a['href']
category = soup.find('span', text='Category').find_next_sibling('span').get_text(strip=True)
asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()


#weighting 
td_elements = soup.find_all('td', class_='pull-right fs-table-alignment')
#len(td_elements) >= 6
sixth_element = td_elements[5]
weighting = sixth_element.get_text(strip=True)


expense_ratio = soup.find('span', text='Expense Ratio').find_next_sibling('span').get_text()

aum = soup.find('span', text='AUM').find_next_sibling('span').get_text()
shares_float = soup.find('span', text='Shares').find_next_sibling('span').get_text()
one_mo_avg_volume = soup.find('span', text='1 Month Avg. Volume').find_next_sibling('span').get_text()
three_mo_avg_volume = soup.find('span', text='3 Month Avg. Volume').find_next_sibling('span').get_text()

#Dividends/Valuation
etf_div = 'https://etfdb.com/etf/EWJ/#etf-ticker-valuation-dividend'
response1 = requests.get(etf_div)
soup1 = BeautifulSoup(response1.text, 'html.parser')

#Dividends
divvy  = soup1.find_all('td', class_='center', attrs={'data-th':'Fund'})


#PE ratio
td_elements_div  = soup1.find_all('div', class_='text-center')
pe_ratio = td_elements_div [1].get_text(strip=True)
peer_pe_ratio = td_elements_div [3].get_text(strip=True)


#Performance
etf_perf_url  = 'https://etfdb.com/etf/EWJ/#performance'
response2 = requests.get(etf_perf_url)
soup2 = BeautifulSoup(response2.text, 'html.parser')

etf_perf = soup2.find_all('tr', {'data-th': 'Fund'}) 
etf_peer = soup2.find_all('td', {'data-th': 'ETF Database Category Average'}) 
performance = soup2.find_all('td', class_='col-lg-4 col-xs-4')

performance[0].get_text(strip=True)

'-0.28%'

In [16]:
etf_div = 'https://etfdb.com/etf/EWJ/#etf-ticker-valuation-dividend'
response1 = requests.get(etf_div)
soup1 = BeautifulSoup(response1.text, 'html.parser')

divvy = soup1.find_all('td', class_='center', attrs={'data-th':'Fund'})
x = divvy[8]
x
# if target_td:
#     dividend_amount = target_td[8].get_text(strip=True)
#     print("Dividend Amount:", dividend_amount)
# else:
#     print("Dividend information not found")

<td class="center" data-th="Fund">
$ 0.43
</td>

In [18]:
target_td[11]

<td class="center" data-th="Fund">
0.91%
</td>

In [ ]:
x[:-4]

In [ ]:
#test
etf_url = 'https://etfdb.com/etf/EWJ/#performance'
response = requests.get(etf_url)
soup = BeautifulSoup(response.text, 'html.parser')

asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()
asset_class_size



'Large-Cap'

In [ ]:
print(ewj[0].get_text(strip=True))
print(ewj[3].get_text(strip=True))
print(ewj[6].get_text(strip=True))
print(ewj[9].get_text(strip=True))
print(ewj[12].get_text(strip=True))
print(ewj[15].get_text(strip=True))

In [ ]:
print(ewj[1].get_text(strip=True))
print(ewj[4].get_text(strip=True))
print(ewj[7].get_text(strip=True))
print(ewj[10].get_text(strip=True))
print(ewj[13].get_text(strip=True))
print(ewj[16].get_text(strip=True))

In [ ]:
print(ewj[10].get_text(strip=True))
print(ewj[11].get_text(strip=True))

print(peer[10].get_text(strip=True))
print(peer[11].get_text(strip=True))

In [ ]:
float(x.replace('$', '').replace(',', '').replace(' M', ''))

In [ ]:
def get_etf_info(etf_tickers):  
    for etf_ticker in etf_tickers:
        etf_url = f'https://etfdb.com/etf/{etf_ticker}/#etf-ticker-profile'
        response = requests.get(etf_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        issuer = soup.find('span', text='Issuer').find_next_sibling('span').get_text()
        brand = soup.find('span', text='Brand').find_next_sibling('span').get_text()
        etf_home_page = soup.find('span', text='ETF Home Page').find_next_sibling('span').a['href']
        category = soup.find('span', text='Category').find_next_sibling('span').get_text(strip=True)
        asset_class_size = soup.find('span', text='Asset Class Size').find_next_sibling('span').get_text()
        
        
        #weighting = soup.find('td', class_='pull-right fs-table-alignment').get_text(strip=True)
        td_elements = soup.find_all('td', class_='pull-right fs-table-alignment')
            #len(td_elements) >= 6
            sixth_element = td_elements[5]
        weighting = sixth_element.get_text(strip=True)
        
        
        expense_ratio = soup.find('span', text='Expense Ratio').find_next_sibling('span').get_text()
        
        aum = soup.find('span', text='AUM').find_next_sibling('span').get_text()
        shares_float = soup.find('span', text='Shares').find_next_sibling('span').get_text()
        one_mo_avg_volume = soup.find('span', text='1 Month Avg. Volume').find_next_sibling('span').get_text()
        three_mo_avg_volume = soup.find('span', text='3 Month Avg. Volume').find_next_sibling('span').get_text()
        

        data_dict = {
            'ETF Ticker': etf_ticker,
            'Issuer': issuer,
            'Brand': brand,
            'ETF Home Page': etf_home_page,
            'Category': category,
            'Asset Class Size': asset_class_size,
            'Weighting': weighting,
            'Expense Ratio': expense_ratio,
            
            'AUM': aum,
            'Shares float': shares_float,
            '1 Month Avg. Volume': one_mo_avg_volume,
            '3 Month Avg. Volume': three_mo_avg_volume,
            
        }
        
        data_list.append(data_dict)
    
    df = pd.DataFrame(data_list)
    
    # Connect to SQLite3 database
    conn = sqlite3.connect('etf_database.db')
    df.to_sql('etf_data', conn, if_exists='replace', index=False)
    conn.close()

# List of ETF tickers
etf_tickers = pd.read_csv('etf_tickers.csv').dropna()
etf_tickers = etf_tickers['ticker'].values.tolist()


# Get ETF information and store in SQLite3 database
get_etf_info(etf_tickers)

In [10]:
#Performance
etf_perf_url  = 'https://etfdb.com/etf/EWJ/#performance'
response2 = requests.get(etf_perf_url)
soup2 = BeautifulSoup(response2.text, 'html.parser')

# etf_perf = soup2.find_all('td', {'data-th': 'Fund'}) 
# etf_peer = soup2.find_all('td', {'data-th': 'ETF Database Category Average'}) 
performance = soup2.find_all('td', class_ ='col-lg-4 col-xs-4')

# <td class="col-lg-4 col-xs-4">
# -0.70%
# </td>

# document.querySelector("#performance-collapse > div > div:nth-child(2) > div > table > tbody > tr:nth-child(1) > td:nth-child(2)")

# performance[0].get_text(strip=True)


#performance#[0].get_text(strip=True)

performance

[<td class="col-lg-4 col-xs-4">
 -0.28%
 </td>,
 <td class="col-lg-4 col-xs-4">
 1.66%
 </td>,
 <td class="col-lg-4 col-xs-4">
 1.07%
 </td>,
 <td class="col-lg-4 col-xs-4">
 3.08%
 </td>,
 <td class="col-lg-4 col-xs-4">
 6.67%
 </td>,
 <td class="col-lg-4 col-xs-4">
 5.78%
 </td>,
 <td class="col-lg-4 col-xs-4">
 13.69%
 </td>,
 <td class="col-lg-4 col-xs-4">
 19.36%
 </td>,
 <td class="col-lg-4 col-xs-4">
 16.10%
 </td>,
 <td class="col-lg-4 col-xs-4">
 10.16%
 </td>,
 <td class="col-lg-4 col-xs-4">
 17.84%
 </td>,
 <td class="col-lg-4 col-xs-4">
 14.13%
 </td>,
 <td class="col-lg-4 col-xs-4">
 4.65%
 </td>,
 <td class="col-lg-4 col-xs-4">
 10.47%
 </td>,
 <td class="col-lg-4 col-xs-4">
 8.42%
 </td>,
 <td class="col-lg-4 col-xs-4">
 2.67%
 </td>,
 <td class="col-lg-4 col-xs-4">
 4.60%
 </td>,
 <td class="col-lg-4 col-xs-4">
 4.33%
 </td>]